### Import packegues

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
from matplotlib.ticker import AutoLocator, AutoMinorLocator, LogLocator
import glob
import cv2
import os
import re

### Define data directories

In [ ]:
data_directory = '/pscratch/sd/b/bigqke/gw170817_global_simulations/cla_sim/92ppEb/cdt_125m_dom_96-96-32km_ncells_192-192-64'
data_directory = '/pscratch/sd/b/bigqke/gw170817_global_simulations/cla_sim/92ppEb/cdt_100m_dom_96-96-32km_ncells_192-192-64'
data_directory = '/pscratch/sd/b/bigqke/gw170817_global_simulations/cla_sim/92ppEb/cdt_250m_dom_96-96-32km_ncells_192-192-64_newprofile'
data_directory = '/pscratch/sd/b/bigqke/gw170817_global_simulations/qke_res_cla_sim/92ppEb/att_01.00e-05_cdt_100m_dom_96-96-32km_ncells_192-192-64'

mesh_files = glob.glob(os.path.join(data_directory, 'mesh_plt*.h5'))
mesh_files_nums = [filenames.split('mesh_plt')[1].split('.h5')[0] for filenames in mesh_files]
mesh_files_sorted = [f for _, f in sorted(zip(mesh_files_nums, mesh_files))]
mesh_files_sorted = np.array(mesh_files_sorted)
nfiles = len(mesh_files_sorted)
print(f'Found {len(mesh_files_sorted)} mesh files.')

### Create mesh

In [ ]:
rho_ye_T_h5py = h5py.File(data_directory+'/rho_Ye_T.hdf5', 'r')

# number of cells
Nx = np.array(rho_ye_T_h5py['/ncellsx'])
Ny = np.array(rho_ye_T_h5py['/ncellsy'])
Nz = np.array(rho_ye_T_h5py['/ncellsz'])
print(f'Nx = {Nx}, Ny = {Ny}, Nz = {Nz}')

# cell size
dx = ( np.array(rho_ye_T_h5py['/xmax_cm']) - np.array(rho_ye_T_h5py['/xmin_cm']) ) / np.array(rho_ye_T_h5py['/ncellsx']) # cm
dy = ( np.array(rho_ye_T_h5py['/ymax_cm']) - np.array(rho_ye_T_h5py['/ymin_cm']) ) / np.array(rho_ye_T_h5py['/ncellsy']) # cm
dz = ( np.array(rho_ye_T_h5py['/zmax_cm']) - np.array(rho_ye_T_h5py['/zmin_cm']) ) / np.array(rho_ye_T_h5py['/ncellsz']) # cm
print(f'dx = {dx} cm, dy = {dy} cm, dz = {dz} cm')

# cell centers
xc = np.linspace(dx / 2, dx * (Nx - 0.5), Nx) # cm
yc = np.linspace(dy / 2, dy * (Ny - 0.5), Ny) # cm
zc = np.linspace(dz / 2, dz * (Nz - 0.5), Nz) # cm

# cell centers mesh
Xc, Yc, Zc = np.meshgrid(xc, yc, zc, indexing='ij') # cm

bh_radius = 5.43e+05 # cm
bh_center_x = 48.0e+5 # cm
bh_center_y = 48.0e+5 # cm
bh_center_z = 16.0e+5 # cm

distance_from_bh = np.sqrt( (Xc - bh_center_x)**2 + (Yc - bh_center_y)**2 + (Zc - bh_center_z)**2 )
mask_bh = (distance_from_bh < bh_radius + 1*np.max([dx,dy,dz]))
mask_x = (Xc < dx) | (Xc > (Nx - 1) * dx)
mask_y = (Yc < dy) | (Yc > (Ny - 1) * dy)
mask_z = (Zc < dz) | (Zc > (Nz - 1) * dz)
mask = mask_bh | mask_x | mask_y | mask_z

rho_ye_T_h5py.close()

### Plot settings and functions

In [ ]:
# Font settings
mpl.rcParams['font.size'] = 22
mpl.rcParams['font.family'] = 'serif'
mpl.rc('text', usetex=False)

# Tick settings
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['xtick.major.pad'] = 8
mpl.rcParams['xtick.minor.size'] = 4

mpl.rcParams['xtick.minor.width'] = 2
mpl.rcParams['ytick.major.size'] = 7
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['ytick.minor.size'] = 4
mpl.rcParams['ytick.minor.width'] = 2

# Axis linewidth
mpl.rcParams['axes.linewidth'] = 2

# Tick direction and enabling ticks on all sides
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True

# Function to apply custom tick locators and other settings to an Axes object
def apply_custom_settings(ax, log_scale_y=False):

    if log_scale_y:
        # Use LogLocator for the y-axis if it's in log scale
        ax.set_yscale('log')
        ax.yaxis.set_major_locator(LogLocator(base=10.0))
        ax.yaxis.set_minor_locator(LogLocator(base=10.0, subs='auto', numticks=100))
    else:
        # Use AutoLocator for regular scales
        ax.yaxis.set_major_locator(AutoLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
    
    # Apply the AutoLocator for the x-axis
    ax.xaxis.set_major_locator(AutoLocator())
    ax.xaxis.set_minor_locator(AutoMinorLocator())

def plot_color_map(x, y, z, bh_r, bh_x, bh_y, min_cb, max_cb, x_label, y_label, title, cbar_label, colormap, filename, doshow=True, dosave=True):

    fig, ax = plt.subplots(figsize=(12, 8))

    # Plot pcolormesh
    # Set non-data points (NaN) to black in the colormap
    cmap = plt.get_cmap(colormap).copy()
    cmap.set_bad(color='black')

    # Plot pcolormesh with masked array to handle NaNs
    c = ax.pcolormesh(x, y, z, shading='auto', cmap=cmap, vmin=min_cb, vmax=max_cb)
    
    # Add contour lines
    # contour = ax.contour(x, y, z, colors='black', linewidths=1.5, levels=3)
    contour = ax.contour(x, y, z, colors='black', linewidths=1.5, levels=[32.0, 32.2, 32.4])
    ax.clabel(contour, inline=True, fontsize=15, fmt='%1.1f')

    # Plot settings
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    # ax.set_title(title+'\nmin: {:.2e}\nmax: {:.2e}'.format(np.nanmin(z), np.nanmax(z)))
    ax.set_title(title)

    # # Plot a black circle at the black hole center
    # circle = plt.Circle(
    #     (bh_x, bh_y),  # center in km
    #     radius=bh_r,  # radius in km
    #     color='black',
    #     fill=True,
    #     linewidth=2
    # )
    # ax.add_artist(circle)

    # Add color bar
    cbar = fig.colorbar(c, ax=ax, label=cbar_label)
    cbar.ax.yaxis.set_minor_locator(AutoMinorLocator())

    apply_custom_settings(ax, False)

    # Ensure equal aspect ratio
    ax.set_aspect('equal', 'box')

    # Save figure
    if dosave:
        fig.savefig(filename, format='png', bbox_inches='tight')

    # Display figure
    if doshow:
        plt.show()
    # display(fig)
    
    # Close figure
    plt.close(fig)

def plot_color_vector_map(x, y, z, bh_r, bh_x, bh_y, vecx, vecy, ve, min_cb, max_cb, x_label, y_label, title, cbar_label, colormap, filename, doshow=True, dosave=True):

    fig, ax = plt.subplots(figsize=(12, 8))


    # Plot pcolormesh
    # Set non-data points (NaN) to black in the colormap
    cmap = plt.get_cmap(colormap).copy()
    cmap.set_bad(color='black')

    # Plot pcolormesh with masked array to handle NaNs
    c = ax.pcolormesh(x, y, z, shading='auto', cmap=cmap, vmin=min_cb, vmax=max_cb)
    
    ax.quiver(x[::ve,::ve], y[::ve,::ve], ve*vecx[::ve,::ve], ve*vecy[::ve,::ve], color='black', scale=1, scale_units='xy', angles='xy')

    # Add contour lines
    # contour = ax.contour(x, y, z, colors='black', linewidths=1.5, levels=3)
    contour = ax.contour(x, y, z, colors='black', linewidths=1.5, levels=[-1.4,-1.0,-0.6])
    ax.clabel(contour, inline=True, fontsize=15, fmt='%1.1f')

    # Plot settings
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    # ax.set_title(title+'\nmin: {:.2e}\nmax: {:.2e}'.format(np.nanmin(z), np.nanmax(z)))
    ax.set_title(title)

    # # Plot a black circle at the black hole center
    # circle = plt.Circle(
    #     (bh_x, bh_y),  # center in km
    #     radius=bh_r,  # radius in km
    #     color='black',
    #     fill=True,
    #     linewidth=2
    # )
    # ax.add_artist(circle)

    # Add color bar
    cbar = fig.colorbar(c, ax=ax, label=cbar_label)
    cbar.ax.yaxis.set_minor_locator(AutoMinorLocator())

    apply_custom_settings(ax, False)

    # Ensure equal aspect ratio
    ax.set_aspect('equal', 'box')

    # Save figure
    if dosave:
        fig.savefig(filename, format='png', bbox_inches='tight')

    # Display figure
    if doshow:
        plt.show()
    # display(fig)
    
    # Close figure
    plt.close(fig)

def plot_vector_map(zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low, time_ms, colorbarlabel, zindx, yindx, filname, minbar, maxbar, doshow=True, dosave=True):

    plot_color_vector_map(  x=Xc[1:-1,1:-1,zindx]/1e5, # km
                            y=Yc[1:-1,1:-1,zindx]/1e5, # km
                            z=zup,
                            bh_r=6.8, # km
                            bh_x=48.0, # km
                            bh_y=48.0, # km
                            vecx=vector_x_up,
                            vecy=vector_y_up,
                            ve=4,
                            min_cb=minbar,
                            max_cb=maxbar, 
                            x_label=r'$x \, (\mathrm{km})$', 
                            y_label=r'$y \, (\mathrm{km})$', 
                            title=fr'$z={Zc[0,0,zindx]/1e5:.1f}\,\mathrm{{km}}$, $t = {time_ms:.2f}\,\mathrm{{ms}}$', 
                            cbar_label=colorbarlabel, 
                            colormap='viridis',
                            filename=f'{filname}_xy.png',
                            doshow=doshow,
                            dosave=dosave)

    plot_color_vector_map(  x=Xc[1:-1,yindx,1:-1]/1e5, 
                            y=Zc[1:-1,yindx,1:-1]/1e5, 
                            z=zlow,
                            bh_r=6.8, # km
                            bh_x=48.0, # km
                            bh_y=16.0, # km
                            vecx=vector_x_low,
                            vecy=vector_y_low,
                            ve=4, 
                            min_cb=minbar,
                            max_cb=maxbar, 
                            x_label=r'$x \, (\mathrm{km})$', 
                            y_label=r'$z \, (\mathrm{km})$', 
                            title=rf'$y={Yc[0,yindx,0]/1e5:.1f}\,\mathrm{{km}}$, $t = {time_ms:.2f}\,\mathrm{{ms}}$', 
                            cbar_label=colorbarlabel, 
                            colormap='viridis',
                            filename=f'{filname}_xz.png',
                            doshow=doshow,
                            dosave=dosave)
    
def plot_up_low(zup, zlow, time_ms, colorbarlabel, zindx, yindx, filname, minbar, maxbar, doshow=True, dosave=True):

    plot_color_map( x=Xc[1:-1,1:-1,zindx]/1e5, # km 
                    y=Yc[1:-1,1:-1,zindx]/1e5, # km 
                    z=zup, 
                    bh_r=6.8, # km
                    bh_x=48.0, # km
                    bh_y=48.0, # km
                    min_cb=minbar,
                    max_cb=maxbar,                    
                    # min_cb=2.27e1,
                    # max_cb=2.55e1,
                    # min_cb=2.22e1,
                    # max_cb=2.69e1,
                    # min_cb=None,
                    # max_cb=None, 
                    x_label=r'$x \, (\mathrm{km})$',
                    y_label=r'$y \, (\mathrm{km})$',
                    title=fr'$z={Zc[0,0,zindx]/1e5:.1f}\,\mathrm{{km}}$, $t = {time_ms:.5f}\,\mathrm{{ms}}$', 
                    cbar_label=colorbarlabel, 
                    colormap='viridis',
                    filename=f'{filname}_xy.png',
                    doshow=doshow,
                    dosave=dosave)

    plot_color_map( x=Xc[1:-1,yindx,1:-1]/1e5, # km
                    y=Zc[1:-1,yindx,1:-1]/1e5, # km
                    z=zlow,
                    bh_r=6.8, # km
                    bh_x=48.0, # km
                    bh_y=16.0, # km
                    min_cb=minbar,
                    max_cb=maxbar,
                    # min_cb=2.27e1,
                    # max_cb=2.55e1,
                    # min_cb=2.22e1,
                    # max_cb=2.69e1, 
                    # min_cb=None,
                    # max_cb=None, 
                    x_label=r'$x \, (\mathrm{km})$', 
                    y_label=r'$z \, (\mathrm{km})$', 
                    title=rf'$y={Yc[0,yindx,0]/1e5:.1f}\,\mathrm{{km}}$, $t = {time_ms:.5f}\,\mathrm{{ms}}$', 
                    cbar_label=colorbarlabel, 
                    colormap='viridis',
                    filename=f'{filname}_xz.png',
                    doshow=doshow,
                    dosave=dosave)
    
def plot_up_low_seismic(zup, zlow, time_ms, colorbarlabel, zindx, yindx, filname, doshow=True, dosave=True):

    plot_color_map( x=Xc[:,:,zindx]/1e5, # km 
                    y=Yc[:,:,zindx]/1e5, # km 
                    z=zup,
                    min_cb=np.nanmax(zup)*-1,
                    max_cb=np.nanmax(zup), 
                    x_label=r'$x \, (\mathrm{km})$',
                    y_label=r'$y \, (\mathrm{km})$',
                    title=fr'$z={Zc[0,0,zindx]/1e5:.1f}\,\mathrm{{km}}$, $t = {time_ms:.2f}\,\mathrm{{ms}}$', 
                    cbar_label=colorbarlabel, 
                    colormap='seismic',
                    filename=f'{filname}_xy.png',
                    doshow=doshow,
                    dosave=dosave)

    plot_color_map( x=Xc[:,yindx,:]/1e5, # km
                    y=Zc[:,yindx,:]/1e5, # km
                    z=zlow,
                    min_cb=np.nanmax(zup)*-1,
                    max_cb=np.nanmax(zup), 
                    x_label=r'$x \, (\mathrm{km})$', 
                    y_label=r'$z \, (\mathrm{km})$', 
                    title=rf'$y={Yc[0,yindx,0]/1e5:.1f}\,\mathrm{{km}}$, $t = {time_ms:.2f}\,\mathrm{{ms}}$', 
                    cbar_label=colorbarlabel, 
                    colormap='seismic',
                    filename=f'{filname}_xz.png',
                    doshow=doshow,
                    dosave=dosave)
    
def plot_color_two_vector_map(x, y, z, vecx1, vecy1, vecx2, vecy2, ve, min_cb, max_cb, x_label, y_label, title, cbar_label, colormap, filename, doshow=True, dosave=True):

    fig, ax = plt.subplots(figsize=(12, 8))

    # Plot pcolormesh
    c = ax.pcolormesh(x, y, z, shading='auto', cmap=colormap, vmin=min_cb, vmax=max_cb)
    
    vecscale = 1.0

    ax.quiver(x[::ve,::ve], y[::ve,::ve], vecscale*ve*vecx2[::ve,::ve], vecscale*ve*vecy2[::ve,::ve], color='white', scale=1.0, scale_units='xy', angles='xy')
    ax.quiver(x[::ve,::ve], y[::ve,::ve], vecscale*ve*vecx1[::ve,::ve], vecscale*ve*vecy1[::ve,::ve], color='black', scale=1.0, scale_units='xy', angles='xy')

    # Add contour lines
    # contour = ax.contour(x, y, z, colors='black', linewidths=1.0, levels=3)
    # ax.clabel(contour, inline=True, fontsize=15, fmt='%1.1e')
    # contour = ax.contour(x, y, z, colors='black', linewidths=0.5, levels=[24, 25, 26])
    # ax.clabel(contour, inline=True, fontsize=15, fmt='%d')

    # Plot settings
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    # ax.set_title(title+'\nmin: {:.2e}\nmax: {:.2e}'.format(np.nanmin(z), np.nanmax(z)))
    ax.set_title(title)

    # Add color bar
    cbar = fig.colorbar(c, ax=ax, label=cbar_label)
    cbar.ax.yaxis.set_minor_locator(AutoMinorLocator())

    apply_custom_settings(ax, False)

    # Ensure equal aspect ratio
    ax.set_aspect('equal', 'box')

    # Save figure
    if dosave:
        fig.savefig(filename, format='png', bbox_inches='tight')

    # Display figure
    if doshow:
        plt.show()
    # display(fig)
    
    # Close figure
    plt.close(fig)

def plot_two_vector_map(zup, zlow, vector_x1_up, vector_y1_up, vector_x2_up, vector_y2_up, vector_x1_low, vector_y1_low, vector_x2_low, vector_y2_low, time_ms, colorbarlabel, zindx, yindx, filname, minbar, maxbar, doshow=True, dosave=True):

    ven = 4

    plot_color_two_vector_map(  x=Xc[:,:,zindx]/1e5, # km
                                y=Yc[:,:,zindx]/1e5, # km
                                z=zup,
                                vecx1=vector_x1_up,
                                vecy1=vector_y1_up,
                                vecx2=vector_x2_up,
                                vecy2=vector_y2_up,
                                ve=ven,
                                min_cb=minbar,
                                max_cb=maxbar, 
                                x_label=r'$x \, (\mathrm{km})$', 
                                y_label=r'$y \, (\mathrm{km})$', 
                                title=fr'$z={Zc[0,0,zindx]/1e5:.1f}\,\mathrm{{km}}$, $t = {time_ms:.2f}\,\mathrm{{ms}}$', 
                                cbar_label=colorbarlabel, 
                                colormap='viridis',
                                filename=f'{filname}_xy.png',
                                doshow=doshow,
                                dosave=dosave)

    plot_color_two_vector_map(  x=Xc[:,yindx,:]/1e5, 
                                y=Zc[:,yindx,:]/1e5, 
                                z=zlow,
                                vecx1=vector_x1_low,
                                vecy1=vector_y1_low,
                                vecx2=vector_x2_low,
                                vecy2=vector_y2_low,
                                ve=ven, 
                                min_cb=minbar,
                                max_cb=maxbar, 
                                x_label=r'$x \, (\mathrm{km})$', 
                                y_label=r'$z \, (\mathrm{km})$', 
                                title=rf'$y={Yc[0,yindx,0]/1e5:.1f}\,\mathrm{{km}}$, $t = {time_ms:.2f}\,\mathrm{{ms}}$', 
                                cbar_label=colorbarlabel, 
                                colormap='viridis',
                                filename=f'{filname}_xz.png',
                                doshow=doshow,
                                dosave=dosave)
    

### Neutrino and antineutrino diagonal number densities

In [ ]:
x_slice_indx = 48 # cell number
y_slice_indx = 48 # cell number
z_slice_indx = 16 # cell number
time_indx    = -1 # last time step

x_slice_indx = 92 # cell number
y_slice_indx = 92 # cell number
z_slice_indx = 32 # cell number
time_indx    = -1 # last time step


# x_slice_indx = 92 # cell number
# y_slice_indx = 107 # cell number
# z_slice_indx = 32 # cell number
# time_indx    = -1 # last time step


# for time_indx in [8,12,16,20,24,28,32,36,40]:
# for time_indx in [1,2,3,4,5,6,7,8,9,10]:

for filepath in [mesh_files_sorted[-1]]:
# for filepath in mesh_files_sorted:

    print(f'Processing file: {filepath}')

    alldata_h5 = h5py.File(filepath, 'r')
    alldata_dict = {key: np.array(alldata_h5[key]) for key in alldata_h5.keys()}
    alldata_h5.close()
    alldata = alldata_dict.copy()
    print("Keys in alldata_dict:", list(alldata.keys()))

    # if time_indx==0:
    if True:
        doshow = True
    else:
        doshow = False

    time_ms = np.array(alldata['t(s)']) * 1.0e3 # ms

    max_cbar = 32.78996125061252
    min_cbar = 28.830254485113596
    # max_cbar = None
    # min_cbar = None
    
    ###############################################
    # Neutrino diagonal terms
    ###############################################

    # # # Electron neutrino number density
    # zup = np.where(mask[1:-1, 1:-1, z_slice_indx], np.nan, np.array(alldata['N00_Re(1|ccm)'])[1:-1, 1:-1, z_slice_indx])
    # zlow = np.where(mask[1:-1, y_slice_indx, 1:-1], np.nan, np.array(alldata['N00_Re(1|ccm)'])[1:-1, y_slice_indx, 1:-1])
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # print("global min:", np.nanmin([np.nanmin(zup), np.nanmin(zlow)]), "global max:", np.nanmax([np.nanmax(zup), np.nanmax(zlow)]))
    # colorbarlabel = fr'$\log ( \, n_{{\nu_{{e}}}}\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # filename=f'./plots/{filepath.split('mesh_plt')[1].split('.h5')[0]}_nee'
    # # mincbar=min_cbar
    # # maxcbar=max_cbar
    # mincbar=None
    # maxcbar=None
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)

    # with h5py.File('./plots/n_e.h5', 'w') as f:
    #     f.create_dataset('xup', data=Xc[1:-1,1:-1,16]/1e5)
    #     f.create_dataset('yup', data=Yc[1:-1,1:-1,16]/1e5)
    #     f.create_dataset('zup', data=zup)
    #     f.create_dataset('xlow', data=Xc[1:-1,48,1:-1]/1e5)
    #     f.create_dataset('ylow', data=Zc[1:-1,48,1:-1]/1e5)
    #     f.create_dataset('zlow', data=zlow)

    # # Muon neutrino number density
    # zup = np.where(mask[1:-1, 1:-1, z_slice_indx], np.nan, np.array(alldata['N11_Re(1|ccm)'])[1:-1, 1:-1, z_slice_indx])
    # zlow = np.where(mask[1:-1, y_slice_indx, 1:-1], np.nan, np.array(alldata['N11_Re(1|ccm)'])[1:-1, y_slice_indx, 1:-1])
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log ( \, n_{{\nu_{{\mu}}}}\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # filename=f'./plots/{filepath.split('mesh_plt')[1].split('.h5')[0]}_nuu'
    # mincbar=None
    # maxcbar=None
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)

    # with h5py.File('./plots/n_x.h5', 'w') as f:
    #     f.create_dataset('xup', data=Xc[1:-1,1:-1,16]/1e5)
    #     f.create_dataset('yup', data=Yc[1:-1,1:-1,16]/1e5)
    #     f.create_dataset('zup', data=zup)
    #     f.create_dataset('xlow', data=Xc[1:-1,48,1:-1]/1e5)
    #     f.create_dataset('ylow', data=Zc[1:-1,48,1:-1]/1e5)
    #     f.create_dataset('zlow', data=zlow)

    # # Tauon neutrino number density
    # zup = np.where(mask[:, :, z_slice_indx], np.nan, np.array(alldata['N22_Re(1|ccm)'])[time_indx, :, :, z_slice_indx])
    # zlow = np.where(mask[:, y_slice_indx, :], np.nan, np.array(alldata['N22_Re(1|ccm)'])[time_indx, :, y_slice_indx, :])
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log ( \, n_{{\nu_{{\tau}}}}\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # filename=f'./plots/{finaldir}_ntt_t_{time_indx}'
    # mincbar=None
    # maxcbar=None
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)

    ###############################################
    # Antineutrino diagonal terms
    ###############################################

    # # Electron antineutrino number density
    # zup = np.where(mask[1:-1, 1:-1, z_slice_indx], np.nan, np.array(alldata['N00_Rebar(1|ccm)'])[1:-1, 1:-1, z_slice_indx])
    # zlow = np.where(mask[1:-1, y_slice_indx, 1:-1], np.nan, np.array(alldata['N00_Rebar(1|ccm)'])[1:-1, y_slice_indx, 1:-1])
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # print("global min:", np.nanmin([np.nanmin(zup), np.nanmin(zlow)]), "global max:", np.nanmax([np.nanmax(zup), np.nanmax(zlow)]))
    # colorbarlabel = fr'$\log ( \, n_{{\bar{{\nu}}_{{e}}}}\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # filename=f'./plots/{filepath.split('mesh_plt')[1].split('.h5')[0]}_nbaree'
    # mincbar=min_cbar
    # maxcbar=max_cbar
    # mincbar=None
    # maxcbar=None
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)

    # with h5py.File('./plots/nbar_e.h5', 'w') as f:
    #     f.create_dataset('xup', data=Xc[1:-1,1:-1,16]/1e5)
    #     f.create_dataset('yup', data=Yc[1:-1,1:-1,16]/1e5)
    #     f.create_dataset('zup', data=zup)
    #     f.create_dataset('xlow', data=Xc[1:-1,48,1:-1]/1e5)
    #     f.create_dataset('ylow', data=Zc[1:-1,48,1:-1]/1e5)
    #     f.create_dataset('zlow', data=zlow)

    # # Muon antineutrino number density
    # zup = np.where(mask[:, :, z_slice_indx], np.nan, np.array(alldata['N11_Rebar(1|ccm)'])[time_indx, :, :, z_slice_indx])
    # zlow = np.where(mask[:, y_slice_indx, :], np.nan, np.array(alldata['N11_Rebar(1|ccm)'])[time_indx, :, y_slice_indx, :])
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log ( \, n_{{\bar{{\nu}}_{{\mu}}}}\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # filename=f'./plots/{finaldir}_nbaruu_t_{time_indx}'
    # mincbar=None
    # maxcbar=None
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)

    # # Tauon antineutrino number density
    # zup = np.where(mask[:, :, z_slice_indx], np.nan, np.array(alldata['N22_Rebar(1|ccm)'])[time_indx, :, :, z_slice_indx])
    # zlow = np.where(mask[:, y_slice_indx, :], np.nan, np.array(alldata['N22_Rebar(1|ccm)'])[time_indx, :, y_slice_indx, :])
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log ( \, n_{{\bar{{\nu}}_{{\tau}}}}\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # filename=f'./plots/{finaldir}_nbartt_t_{time_indx}'
    # mincbar=None
    # maxcbar=None
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)

    ###############################################
    # All heavies neutrinos number density toguether
    ###############################################

    # # Heavy neutrino number density
    # zup = np.where(mask[1:-1, 1:-1, z_slice_indx], np.nan, np.array(alldata['N11_Re(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx])
    # zlow = np.where(mask[1:-1, y_slice_indx, 1:-1], np.nan, np.array(alldata['N11_Re(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1])
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # print("global min:", np.nanmin([np.nanmin(zup), np.nanmin(zlow)]), "global max:", np.nanmax([np.nanmax(zup), np.nanmax(zlow)]))
    # colorbarlabel = fr'$\log ( \, n_{{ \, \nu_{{x}} }} \, [\mathrm{{cm}}^{{-3}}] \, )$'
    # filename=f'./plots/{finaldir}_nxx_t_{time_indx}'
    # mincbar=min_cbar
    # maxcbar=max_cbar
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)




### Neutrino and antineutrino off-diagonal number densities

In [ ]:
x_slice_indx = 96 # cell number
y_slice_indx = 96 # cell number
z_slice_indx = 32 # cell number
time_indx    = -1 # last time step

# for filepath in mesh_files_sorted[::100]:
# for filepath in [mesh_files_sorted[20], mesh_files_sorted[70],mesh_files_sorted[-1]]:
for filepath in [mesh_files_sorted[-1]]:
# for filepath in mesh_files_sorted:
# for filepath in mesh_files_sorted[613:]:

    print(f'Processing file: {filepath}')

    alldata_h5 = h5py.File(filepath, 'r')
    alldata_dict = {key: np.array(alldata_h5[key]) for key in alldata_h5.keys()}
    alldata_h5.close()
    alldata = alldata_dict.copy()
    print("Keys in alldata_dict:", list(alldata.keys()))

    # if time_indx==0:
    if True:
        doshow = True
    else:
        doshow = False

    time_ms = np.array(alldata['t(s)']) * 1.0e3 # ms
    print("time_ms=", time_ms)

    mincbar= 24.04248312959716 
    maxcbar= 27.091413671485224

    ###############################################
    # Neutrino off diagonal terms
    ###############################################

    # # Magnitud of electron-muon mixing neutrino number density
    # zup = np.where(mask[1:-1, 1:-1, z_slice_indx], np.nan, np.sqrt(np.array(alldata['N01_Re(1|ccm)'])[1:-1, 1:-1, z_slice_indx]**2+np.array(alldata['N01_Im(1|ccm)'])[1:-1, 1:-1, z_slice_indx]**2))
    # zlow = np.where(mask[1:-1, y_slice_indx, 1:-1], np.nan, np.sqrt(np.array(alldata['N01_Re(1|ccm)'])[1:-1, y_slice_indx, 1:-1]**2+np.array(alldata['N01_Im(1|ccm)'])[1:-1, y_slice_indx, 1:-1]**2))
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,n_{{\nu_{{e}}\nu_{{\mu}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # filename=f'./plots/{filepath.split('mesh_plt')[1].split('.h5')[0]}_neu'
    # print("global min:", np.nanmin([np.nanmin(zup[np.isfinite(zup)]), np.nanmin(zlow[np.isfinite(zlow)])]), 
    #       "global max:", np.nanmax([np.nanmax(zup[np.isfinite(zup)]), np.nanmax(zlow[np.isfinite(zlow)])]))
    # print("mincbar=", np.nanmin(zlow[np.isfinite(zlow)]), "\nmaxcbar=", np.nanmax(zlow[np.isfinite(zlow)]))
    # mincbar=None
    # maxcbar=None
    # # mincbar=22.2
    # # maxcbar=27
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)

    # with h5py.File(f'./plots/n_eu_{filepath.split('mesh_plt')[1].split('.h5')[0]}.h5', 'w') as f:
    #     f.create_dataset('xup', data=Xc[1:-1,1:-1,32]/1e5)
    #     f.create_dataset('yup', data=Yc[1:-1,1:-1,32]/1e5)
    #     f.create_dataset('zup', data=zup)
    #     f.create_dataset('xlow', data=Xc[1:-1,96,1:-1]/1e5)
    #     f.create_dataset('ylow', data=Zc[1:-1,96,1:-1]/1e5)
    #     f.create_dataset('zlow', data=zlow)

    # # Magnitud of electron-tauon mixing neutrino number density
    # zup = np.where(mask[1:-1, 1:-1, z_slice_indx], np.nan, np.sqrt(np.array(alldata['N02_Re(1|ccm)'])[1:-1, 1:-1, z_slice_indx]**2+np.array(alldata['N02_Im(1|ccm)'])[1:-1, 1:-1, z_slice_indx]**2))
    # zlow = np.where(mask[1:-1, y_slice_indx, 1:-1], np.nan, np.sqrt(np.array(alldata['N02_Re(1|ccm)'])[1:-1, y_slice_indx, 1:-1]**2+np.array(alldata['N02_Im(1|ccm)'])[1:-1, y_slice_indx, 1:-1]**2))
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # # print("global min:", np.nanmin([np.nanmin(zup[np.isfinite(zup)]), np.nanmin(zlow[np.isfinite(zlow)])]), 
    # #       "global max:", np.nanmax([np.nanmax(zup[np.isfinite(zup)]), np.nanmax(zlow[np.isfinite(zlow)])]))
    # print("mincbar=", np.nanmin(zlow[np.isfinite(zlow)]), "\nmaxcbar=", np.nanmax(zlow[np.isfinite(zlow)]))
    # colorbarlabel = fr'$\log (\,|\,n_{{\nu_{{e}}\nu_{{\tau}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # filename=f'./plots/{filepath.split('mesh_plt')[1].split('.h5')[0]}_net'
    # mincbar=None
    # maxcbar=None
    # # mincbar= 23.580158260244104 
    # # maxcbar= 28.323431144400264
    # # maxcbar=30 
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)

    # # Magnitud of muon-tauon mixing neutrino number density
    # zup = np.where(mask[1:-1, 1:-1, z_slice_indx], np.nan, np.sqrt(np.array(alldata['N12_Re(1|ccm)'])[1:-1, 1:-1, z_slice_indx]**2+np.array(alldata['N12_Im(1|ccm)'])[1:-1, 1:-1, z_slice_indx]**2))
    # zlow = np.where(mask[1:-1, y_slice_indx, 1:-1], np.nan, np.sqrt(np.array(alldata['N12_Re(1|ccm)'])[1:-1, y_slice_indx, 1:-1]**2+np.array(alldata['N12_Im(1|ccm)'])[1:-1, y_slice_indx, 1:-1]**2))
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,n_{{\nu_{{\mu}}\nu_{{\tau}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # filename=f'./plots/{filepath.split('mesh_plt')[1].split('.h5')[0]}_nut'
    # mincbar=None
    # maxcbar=None
    # # mincbar=17.0
    # # maxcbar=20.3
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)

    ###############################################
    # Antieutrino off diagonal terms
    ###############################################

    # # Magnitud of electron-muon mixing antineutrino number density
    # zup = np.where(mask[1:-1, 1:-1, z_slice_indx], np.nan, np.sqrt(np.array(alldata['N01_Rebar(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2+np.array(alldata['N01_Imbar(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2))
    # zlow = np.where(mask[1:-1, y_slice_indx, 1:-1], np.nan, np.sqrt(np.array(alldata['N01_Rebar(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2+np.array(alldata['N01_Imbar(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2))
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,n_{{\bar{{\nu}}_{{e}}\bar{{\nu}}_{{\mu}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # filename=f'./plots/{finaldir}_nbareu_t_{time_indx}'
    # mincbar=None
    # maxcbar=None
    # # mincbar=21.6
    # # maxcbar=25.2
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)
    
    # # Magnitud of electron-tauon mixing antineutrino number density
    # zup = np.where(mask[1:-1, 1:-1, z_slice_indx], np.nan, np.sqrt(np.array(alldata['N02_Rebar(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2+np.array(alldata['N02_Imbar(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2))
    # zlow = np.where(mask[1:-1, y_slice_indx, 1:-1], np.nan, np.sqrt(np.array(alldata['N02_Rebar(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2+np.array(alldata['N02_Imbar(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2))
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,n_{{\bar{{\nu}}_{{e}}\bar{{\nu}}_{{\tau}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # filename=f'./plots/{finaldir}_nbaret_t_{time_indx}'
    # mincbar=None
    # maxcbar=None
    # # mincbar=23.0
    # # maxcbar=26.6
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)

    # # Magnitud of muon-tauon mixing antineutrino number density
    # zup = np.where(mask[1:-1, 1:-1, z_slice_indx], np.nan, np.sqrt(np.array(alldata['N12_Rebar(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2+np.array(alldata['N12_Imbar(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2))
    # zlow = np.where(mask[1:-1, y_slice_indx, 1:-1], np.nan, np.sqrt(np.array(alldata['N12_Rebar(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2+np.array(alldata['N12_Imbar(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2))
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,n_{{\bar{{\nu}}_{{\mu}}\bar{{\nu}}_{{\tau}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # filename=f'./plots/{finaldir}_nbarut_t_{time_indx}'
    # mincbar=None
    # maxcbar=None
    # # mincbar=16.7
    # # maxcbar=19.9
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)

### PLOT DENSITY MATRICES

In [ ]:
x_slice_indx = 96 # cell number
y_slice_indx = 96 # cell number
z_slice_indx = 32 # cell number
time_indx    = -1 # last time step

print(f'mesh_files_sorted[-1]={mesh_files_sorted[-1]}')
alldatafin_h5 = h5py.File(mesh_files_sorted[-1], 'r')
alldatafin_dict = {key: np.array(alldatafin_h5[key]) for key in alldatafin_h5.keys()}
alldatafin_h5.close()
alldatafin = alldatafin_dict.copy()
zup_prev_fin  = np.sqrt(np.array(alldatafin['N01_Re(1|ccm)'])[1:-1, 1:-1, z_slice_indx]**2+np.array(alldatafin['N01_Im(1|ccm)'])[1:-1, 1:-1, z_slice_indx]**2) / ( np.array(alldatafin['N00_Re(1|ccm)'])[1:-1, 1:-1, z_slice_indx] + np.array(alldatafin['N11_Re(1|ccm)'])[1:-1, 1:-1, z_slice_indx] + np.array(alldatafin['N22_Re(1|ccm)'])[1:-1, 1:-1, z_slice_indx] )
zlow_prev_fin = np.sqrt(np.array(alldatafin['N01_Re(1|ccm)'])[1:-1, y_slice_indx, 1:-1]**2+np.array(alldatafin['N01_Im(1|ccm)'])[1:-1, y_slice_indx, 1:-1]**2) / ( np.array(alldatafin['N00_Re(1|ccm)'])[1:-1, y_slice_indx, 1:-1] + np.array(alldatafin['N11_Re(1|ccm)'])[1:-1, y_slice_indx, 1:-1] + np.array(alldatafin['N22_Re(1|ccm)'])[1:-1, y_slice_indx, 1:-1] )

# for filepath in mesh_files_sorted[::100]:
# for filepath in [mesh_files_sorted[200],mesh_files_sorted[-1]]:
# for filepath in mesh_files_sorted:
for filepath in mesh_files_sorted[279:]:

    print(f'Processing file: {filepath}')

    alldata_h5 = h5py.File(filepath, 'r')
    alldata_dict = {key: np.array(alldata_h5[key]) for key in alldata_h5.keys()}
    alldata_h5.close()
    alldata = alldata_dict.copy()
    print("Keys in alldata_dict:", list(alldata.keys()))

    # if time_indx==0:
    if True:
        doshow = False
    else:
        doshow = False

    time_ms = np.array(alldata['t(s)']) * 1.0e3 # ms
    print("time_ms=", time_ms)

    ###############################################
    # Neutrino off diagonal terms
    ###############################################

    # # Magnitud of electron-muon mixing neutrino number density
    # zup_prev  = np.sqrt(np.array(alldata['N01_Re(1|ccm)'])[1:-1, 1:-1, z_slice_indx]**2+np.array(alldata['N01_Im(1|ccm)'])[1:-1, 1:-1, z_slice_indx]**2) / ( np.array(alldata['N00_Re(1|ccm)'])[1:-1, 1:-1, z_slice_indx] + np.array(alldata['N11_Re(1|ccm)'])[1:-1, 1:-1, z_slice_indx] + np.array(alldata['N22_Re(1|ccm)'])[1:-1, 1:-1, z_slice_indx] )
    # zlow_prev = np.sqrt(np.array(alldata['N01_Re(1|ccm)'])[1:-1, y_slice_indx, 1:-1]**2+np.array(alldata['N01_Im(1|ccm)'])[1:-1, y_slice_indx, 1:-1]**2) / ( np.array(alldata['N00_Re(1|ccm)'])[1:-1, y_slice_indx, 1:-1] + np.array(alldata['N11_Re(1|ccm)'])[1:-1, y_slice_indx, 1:-1] + np.array(alldata['N22_Re(1|ccm)'])[1:-1, y_slice_indx, 1:-1] )
    # # max_value = max( np.nanmax(zup_prev[np.isfinite(zup_prev)]), np.nanmax(zlow_prev[np.isfinite(zlow_prev)]) )
    # # min_value = min( np.nanmin(zup_prev[np.isfinite(zup_prev)]), np.nanmin(zlow_prev[np.isfinite(zlow_prev)]) )
    # # print("max_value=", np.log10(max_value))
    # # print("min_value=", np.log10(min_value))
    # maxcbar = -5.1157205194354765
    # mincbar = -8.914617831024938
    # zup = np.where(mask[1:-1, 1:-1, z_slice_indx], np.nan, zup_prev)
    # zlow = np.where(mask[1:-1, y_slice_indx, 1:-1], np.nan, zlow_prev)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,\rho_{{\nu_{{e}}\nu_{{\mu}}}}\,|\,)$'
    # filename=f'./plots/{filepath.split('mesh_plt')[1].split('.h5')[0]}_rho_eu'
    # # mincbar=None
    # # maxcbar=None
    # # mincbar=22.2
    # # maxcbar=27
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)

    # # Magnitud of electron-muon mixing neutrino number density
    # zup_prev  = np.sqrt(np.array(alldata['N01_Re(1|ccm)'])[1:-1, 1:-1, z_slice_indx]**2+np.array(alldata['N01_Im(1|ccm)'])[1:-1, 1:-1, z_slice_indx]**2) / ( np.array(alldata['N00_Re(1|ccm)'])[1:-1, 1:-1, z_slice_indx] + np.array(alldata['N11_Re(1|ccm)'])[1:-1, 1:-1, z_slice_indx] + np.array(alldata['N22_Re(1|ccm)'])[1:-1, 1:-1, z_slice_indx] )
    # zlow_prev = np.sqrt(np.array(alldata['N01_Re(1|ccm)'])[1:-1, y_slice_indx, 1:-1]**2+np.array(alldata['N01_Im(1|ccm)'])[1:-1, y_slice_indx, 1:-1]**2) / ( np.array(alldata['N00_Re(1|ccm)'])[1:-1, y_slice_indx, 1:-1] + np.array(alldata['N11_Re(1|ccm)'])[1:-1, y_slice_indx, 1:-1] + np.array(alldata['N22_Re(1|ccm)'])[1:-1, y_slice_indx, 1:-1] )
    # zup_prev  = zup_prev - zup_prev_fin
    # zlow_prev = zlow_prev - zlow_prev_fin
    # # max_value = max( np.nanmax(zup_prev[np.isfinite(zup_prev)]), np.nanmax(zlow_prev[np.isfinite(zlow_prev)]) )
    # # min_value = min( np.nanmin(zup_prev[np.isfinite(zup_prev)]), np.nanmin(zlow_prev[np.isfinite(zlow_prev)]) )
    # # print("max_value=", max_value)
    # # print("min_value=", min_value)
    # zup = np.where(mask[1:-1, 1:-1, z_slice_indx], np.nan, zup_prev)
    # zlow = np.where(mask[1:-1, y_slice_indx, 1:-1], np.nan, zlow_prev)
    # colorbarlabel = fr'$\rho_{{\nu_{{e}}\nu_{{\mu}}}} - \rho_{{\nu_{{e}}\nu_{{\mu}}}}^f$'
    # filename=f'./plots/{filepath.split('mesh_plt')[1].split('.h5')[0]}_delta_rho_eu'
    # # maxcbar= +3.819189911368927e-06
    # # mincbar= -3.819189911368927e-06
    # maxcbar= +1e-7
    # mincbar= -1e-7
    # # mincbar=None
    # # maxcbar=None
    # # mincbar=22.2
    # # maxcbar=27
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)

### Plots that require seismic scale

In [ ]:
# for time_indx in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]:
for time_indx in [-1]:
    
    if time_indx==-1:
        doshow = True
    else:
        doshow = False

    time_ms = np.array(alldata['t(s)']) * 1.0e3 # ms

    # # Electron neutrino minus antineutrinos number density
    # zup  = np.where(mask[:, :, z_slice_indx], np.nan, np.array(alldata['N00_Re(1|ccm)'])[time_indx, :, :, z_slice_indx]-np.array(alldata['N00_Rebar(1|ccm)'])[time_indx, :, :, z_slice_indx])
    # zlow = np.where(mask[:, y_slice_indx, :], np.nan, np.array(alldata['N00_Re(1|ccm)'])[time_indx, :, y_slice_indx, :]-np.array(alldata['N00_Rebar(1|ccm)'])[time_indx, :, y_slice_indx, :])
    # colorbarlabel = fr'$n_{{\nu_{{e}}}}-n_{{\bar{{\nu}}_{{e}}}}$ ($\mathrm{{cm}}^{{-3}}$)'
    # filename=f'./plots/{finaldir}_nee_minus_nbaree_t_{time_indx}'
    # plot_up_low_seismic(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, doshow=doshow, dosave=True)

### Function to compute vector field componets

In [ ]:
def compute_vector_field_components(vector_grid, z_slice_indx, y_slice_indx, mask):

    quantity_up = np.sqrt(np.sum(vector_grid[:, :, :, z_slice_indx]**2, axis=0))
    zup = np.where(mask[:, :, z_slice_indx], np.nan, quantity_up)

    quantity_low = np.sqrt(np.sum(vector_grid[:, :, y_slice_indx, :]**2, axis=0))
    zlow = np.where(mask[:, y_slice_indx, :], np.nan, quantity_low)

    vector_x_up = vector_grid[0, :, :, z_slice_indx] / zup
    vector_y_up = vector_grid[1, :, :, z_slice_indx] / zup
 
    vector_x_low = vector_grid[0, :, y_slice_indx, :] / zlow
    vector_y_low = vector_grid[2, :, y_slice_indx, :] / zlow

    return zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low

### Color maps with arrows: Neutrino and antineutrino diagonal fluxes

In [ ]:
x_slice_indx = 96 # cell number
y_slice_indx = 96 # cell number
z_slice_indx = 32 # cell number
time_indx    = -1 # last time step

# for filepath in mesh_files_sorted[::100]:
for filepath in [mesh_files_sorted[20], mesh_files_sorted[70],mesh_files_sorted[-1]]:
# for filepath in mesh_files_sorted:
# for filepath in mesh_files_sorted[613:]:

    print(f'Processing file: {filepath}')

    alldata_h5 = h5py.File(filepath, 'r')
    alldata_dict = {key: np.array(alldata_h5[key]) for key in alldata_h5.keys()}
    alldata_h5.close()
    alldata = alldata_dict.copy()
    print("Keys in alldata_dict:", list(alldata.keys()))

    # if time_indx==0:
    if True:
        doshow = True
    else:
        doshow = False

    time_ms = np.array(alldata['t(s)']) * 1.0e3 # ms
    print("time_ms=", time_ms)

    ##############################################
    # Neutrino diagonal flux terms
    ##############################################

    # # Electron neutrino number flux density
    # vectorfield = np.array([np.array(alldata['Fx00_Re(1|ccm)']), # x component
    #                         np.array(alldata['Fy00_Re(1|ccm)']), # y component
    #                         np.array(alldata['Fz00_Re(1|ccm)'])]) # z component
    # zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low = compute_vector_field_components(vectorfield, z_slice_indx, y_slice_indx, mask)
    # zup = zup[1:-1,1:-1]
    # zlow = zlow[1:-1,1:-1]
    # vector_x_up = vector_x_up[1:-1,1:-1]
    # vector_y_up = vector_y_up[1:-1,1:-1]
    # vector_x_low = vector_x_low[1:-1,1:-1]
    # vector_y_low = vector_y_low[1:-1,1:-1]
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\nu_{{e}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # mincbar=None
    # maxcbar=None
    # # mincbar=23.0
    # # maxcbar=26.6
    # filename=f'./plots/{filepath.split('mesh_plt')[1].split('.h5')[0]}_fee_t_{time_indx}'
    # plot_vector_map(zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)

    # with h5py.File('./plots/f_e.h5', 'w') as f:
    #     f.create_dataset('xup', data=Xc[1:-1,1:-1,16]/1e5)
    #     f.create_dataset('yup', data=Yc[1:-1,1:-1,16]/1e5)
    #     f.create_dataset('zup', data=zup)
    #     f.create_dataset('vector_x_up', data=vector_x_up)
    #     f.create_dataset('vector_y_up', data=vector_y_up)
    #     f.create_dataset('xlow', data=Xc[1:-1,48,1:-1]/1e5)
    #     f.create_dataset('ylow', data=Zc[1:-1,48,1:-1]/1e5)
    #     f.create_dataset('zlow', data=zlow)
    #     f.create_dataset('vector_x_low', data=vector_x_low)
    #     f.create_dataset('vector_y_low', data=vector_y_low)

    # # Muon neutrino number flux density
    # vectorfield = np.array([np.array(alldata['Fx11_Re(1|ccm)']), # x component
    #                         np.array(alldata['Fy11_Re(1|ccm)']), # y component
    #                         np.array(alldata['Fz11_Re(1|ccm)'])]) # z component
    # zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low = compute_vector_field_components(vectorfield, time_indx, z_slice_indx, y_slice_indx, mask)
    # zup = zup[1:-1,1:-1]
    # zlow = zlow[1:-1,1:-1]
    # vector_x_up = vector_x_up[1:-1,1:-1]
    # vector_y_up = vector_y_up[1:-1,1:-1]
    # vector_x_low = vector_x_low[1:-1,1:-1]
    # vector_y_low = vector_y_low[1:-1,1:-1]
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\nu_{{\mu}}\nu_{{\mu}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # mincbar=None
    # maxcbar=None
    # # mincbar=23.0
    # # maxcbar=26.6
    # filename=f'./plots/{finaldir}_fuu_t_{time_indx}'
    # plot_vector_map(zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)

    # with h5py.File('./plots/f_x.h5', 'w') as f:
    #     f.create_dataset('xup', data=Xc[1:-1,1:-1,16]/1e5)
    #     f.create_dataset('yup', data=Yc[1:-1,1:-1,16]/1e5)
    #     f.create_dataset('zup', data=zup)
    #     f.create_dataset('vector_x_up', data=vector_x_up)
    #     f.create_dataset('vector_y_up', data=vector_y_up)
    #     f.create_dataset('xlow', data=Xc[1:-1,48,1:-1]/1e5)
    #     f.create_dataset('ylow', data=Zc[1:-1,48,1:-1]/1e5)
    #     f.create_dataset('zlow', data=zlow)
    #     f.create_dataset('vector_x_low', data=vector_x_low)
    #     f.create_dataset('vector_y_low', data=vector_y_low)

    # # Tauon neutrino number flux density
    # vectorfield = np.array([np.array(alldata['Fx22_Re(1|ccm)']), # x component
    #                         np.array(alldata['Fy22_Re(1|ccm)']), # y component
    #                         np.array(alldata['Fz22_Re(1|ccm)'])]) # z component
    # zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low = compute_vector_field_components(vectorfield, time_indx, z_slice_indx, y_slice_indx, mask)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\nu_{{\tau}}\nu_{{\tau}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # mincbar=None
    # maxcbar=None
    # # mincbar=23.0
    # # maxcbar=26.6
    # filename=f'./plots/{finaldir}_ftt_t_{time_indx}'
    # plot_vector_map(zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)

    ##############################################
    # Antineutrino diagonal flux terms
    ##############################################

    # # Electron antineutrino number flux density
    # vectorfield = np.array([np.array(alldata['Fx00_Rebar(1|ccm)']), # x component
    #                         np.array(alldata['Fy00_Rebar(1|ccm)']), # y component
    #                         np.array(alldata['Fz00_Rebar(1|ccm)'])]) # z component
    # zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low = compute_vector_field_components(vectorfield, time_indx, z_slice_indx, y_slice_indx, mask)
    # zup = zup[1:-1,1:-1]
    # zlow = zlow[1:-1,1:-1]
    # vector_x_up = vector_x_up[1:-1,1:-1]
    # vector_y_up = vector_y_up[1:-1,1:-1]
    # vector_x_low = vector_x_low[1:-1,1:-1]
    # vector_y_low = vector_y_low[1:-1,1:-1]
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\bar{{\nu}}_{{e}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # mincbar=None
    # maxcbar=None
    # # mincbar=23.0
    # # maxcbar=26.6
    # filename=f'./plots/{finaldir}_fbaree_t_{time_indx}'
    # plot_vector_map(zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)

    # with h5py.File('./plots/fbar_e.h5', 'w') as f:
    #     f.create_dataset('xup', data=Xc[1:-1,1:-1,16]/1e5)
    #     f.create_dataset('yup', data=Yc[1:-1,1:-1,16]/1e5)
    #     f.create_dataset('zup', data=zup)
    #     f.create_dataset('vector_x_up', data=vector_x_up)
    #     f.create_dataset('vector_y_up', data=vector_y_up)
    #     f.create_dataset('xlow', data=Xc[1:-1,48,1:-1]/1e5)
    #     f.create_dataset('ylow', data=Zc[1:-1,48,1:-1]/1e5)
    #     f.create_dataset('zlow', data=zlow)
    #     f.create_dataset('vector_x_low', data=vector_x_low)
    #     f.create_dataset('vector_y_low', data=vector_y_low)

    # # Muon antineutrino number flux density
    # vectorfield = np.array([np.array(alldata['Fx11_Rebar(1|ccm)']), # x component
    #                         np.array(alldata['Fy11_Rebar(1|ccm)']), # y component
    #                         np.array(alldata['Fz11_Rebar(1|ccm)'])]) # z component
    # zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low = compute_vector_field_components(vectorfield, time_indx, z_slice_indx, y_slice_indx, mask)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\bar{{\nu}}_{{\mu}}\bar{{\nu}}_{{\mu}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # mincbar=None
    # maxcbar=None
    # # mincbar=23.0
    # # maxcbar=26.6
    # filename=f'./plots/{finaldir}_fbaruu_t_{time_indx}'
    # plot_vector_map(zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)

    # # Tauon antineutrino number flux density
    # vectorfield = np.array([np.array(alldata['Fx22_Rebar(1|ccm)']), # x component
    #                         np.array(alldata['Fy22_Rebar(1|ccm)']), # y component
    #                         np.array(alldata['Fz22_Rebar(1|ccm)'])]) # z component
    # zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low = compute_vector_field_components(vectorfield, time_indx, z_slice_indx, y_slice_indx, mask)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\bar{{\nu}}_{{\tau}}\bar{{\nu}}_{{\tau}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # mincbar=None
    # maxcbar=None
    # # mincbar=23.0
    # # maxcbar=26.6
    # filename=f'./plots/{finaldir}_fbartt_t_{time_indx}'
    # plot_vector_map(zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)

    ###############################################
    # All heavies flux number density toguether
    ###############################################

    # # Heavy neutrino number flux density
    # vectorfield = np.array([np.array(alldata['Fx11_Re(1|ccm)']), # x component
    #                         np.array(alldata['Fy11_Re(1|ccm)']), # y component
    #                         np.array(alldata['Fz11_Re(1|ccm)'])]) # z component
    # zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low = compute_vector_field_components(vectorfield, time_indx, z_slice_indx, y_slice_indx, mask)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{ \, \nu_{{\mu}} ,\, \bar{{\nu}}_{{\mu}} ,\, \nu_{{\tau}} ,\, \bar{{\nu}}_{{\tau}} }}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # mincbar=None
    # maxcbar=None
    # # mincbar=23.0
    # # maxcbar=26.6
    # filename=f'./plots/{finaldir}_fxx_t_{time_indx}'
    # plot_vector_map(zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)

### Color maps with arrows: Neutrino and antineutrino diagonal flux factors

In [ ]:
x_slice_indx = 96 # cell number
y_slice_indx = 96 # cell number
z_slice_indx = 32 # cell number
time_indx    = -1 # last time step

# for filepath in mesh_files_sorted[::100]:
# for filepath in [mesh_files_sorted[20], mesh_files_sorted[70],mesh_files_sorted[-1]]:
for filepath in [mesh_files_sorted[-1]]:
# for filepath in mesh_files_sorted:
# for filepath in mesh_files_sorted[613:]:

    print(f'Processing file: {filepath}')

    alldata_h5 = h5py.File(filepath, 'r')
    alldata_dict = {key: np.array(alldata_h5[key]) for key in alldata_h5.keys()}
    alldata_h5.close()
    alldata = alldata_dict.copy()
    print("Keys in alldata_dict:", list(alldata.keys()))

    # if time_indx==0:
    if True:
        doshow = True
    else:
        doshow = False

    time_ms = np.array(alldata['t(s)']) * 1.0e3 # ms
    print("time_ms=", time_ms)

    min_cbar = -2.6136778444545605
    max_cbar = -0.05781070037191741

    ##############################################
    # Neutrino diagonal flux terms
    ##############################################

    # # Electron neutrino number flux density
    # fx = np.array(alldata['Fx00_Re(1|ccm)'])  # x component
    # fy = np.array(alldata['Fy00_Re(1|ccm)'])  # y component
    # fz = np.array(alldata['Fz00_Re(1|ccm)'])  # z component
    # n  = np.array(alldata['N00_Re(1|ccm)'])   # normalization
    # vectorfield = np.array([
    #     fx / n,
    #     fy / n,
    #     fz / n
    # ])
    # zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low = compute_vector_field_components(vectorfield, z_slice_indx, y_slice_indx, mask)
    # vector_x_up = vector_x_up[1:-1, 1:-1]
    # vector_y_up = vector_y_up[1:-1, 1:-1]
    # vector_x_low = vector_x_low[1:-1, 1:-1]
    # vector_y_low = vector_y_low[1:-1, 1:-1]
    # print("vector_x_up.shape", vector_x_up.shape)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # zup = zup[1:-1, 1:-1]
    # zlow = zlow[1:-1, 1:-1]
    # print("global min:", np.nanmin([np.nanmin(zup), np.nanmin(zlow)]), "global max:", np.nanmax([np.nanmax(zup), np.nanmax(zlow)]))
    # colorbarlabel = fr'$\log ( \, | \, \vec{{f}}_{{\nu_{{e}}}} \, / \, n_{{\nu_{{e}}}} \, | \, )$'
    # mincbar=min_cbar
    # maxcbar=max_cbar
    # mincbar=None
    # maxcbar=None
    # filename=f'./plots/{filepath.split('mesh_plt')[1].split('.h5')[0]}_fhateu'
    # plot_vector_map(zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)

    # with h5py.File('./plots/f_e.h5', 'w') as f:
    #     f.create_dataset('xup', data=Xc[1:-1,1:-1,16]/1e5)
    #     f.create_dataset('yup', data=Yc[1:-1,1:-1,16]/1e5)
    #     f.create_dataset('zup', data=zup)
    #     f.create_dataset('vector_x_up', data=vector_x_up)
    #     f.create_dataset('vector_y_up', data=vector_y_up)
    #     f.create_dataset('xlow', data=Xc[1:-1,48,1:-1]/1e5)
    #     f.create_dataset('ylow', data=Zc[1:-1,48,1:-1]/1e5)
    #     f.create_dataset('zlow', data=zlow)
    #     f.create_dataset('vector_x_low', data=vector_x_low)
    #     f.create_dataset('vector_y_low', data=vector_y_low)

    # # Muon neutrino number flux density
    # vectorfield = np.array([np.array(alldata['Fx11_Re(1|ccm)']), # x component
    #                         np.array(alldata['Fy11_Re(1|ccm)']), # y component
    #                         np.array(alldata['Fz11_Re(1|ccm)'])]) # z component
    # zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low = compute_vector_field_components(vectorfield, time_indx, z_slice_indx, y_slice_indx, mask)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\nu_{{\mu}}\nu_{{\mu}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # mincbar=None
    # maxcbar=None
    # # mincbar=23.0
    # # maxcbar=26.6
    # filename=f'./plots/{finaldir}_fuu_t_{time_indx}'
    # plot_vector_map(zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)

    # # Tauon neutrino number flux density
    # vectorfield = np.array([np.array(alldata['Fx22_Re(1|ccm)']), # x component
    #                         np.array(alldata['Fy22_Re(1|ccm)']), # y component
    #                         np.array(alldata['Fz22_Re(1|ccm)'])]) # z component
    # zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low = compute_vector_field_components(vectorfield, time_indx, z_slice_indx, y_slice_indx, mask)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\nu_{{\tau}}\nu_{{\tau}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # mincbar=None
    # maxcbar=None
    # # mincbar=23.0
    # # maxcbar=26.6
    # filename=f'./plots/{finaldir}_ftt_t_{time_indx}'
    # plot_vector_map(zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)

    ##############################################
    # Antineutrino diagonal flux terms
    ##############################################

    # # Electron antineutrino number flux density
    # fx = np.array(alldata['Fx00_Rebar(1|ccm)'])  # x component
    # fy = np.array(alldata['Fy00_Rebar(1|ccm)'])  # y component
    # fz = np.array(alldata['Fz00_Rebar(1|ccm)'])  # z component
    # n  = np.array(alldata['N00_Rebar(1|ccm)'])   # normalization
    # vectorfield = np.array([
    #     fx / n,
    #     fy / n,
    #     fz / n
    # ])
    # zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low = compute_vector_field_components(vectorfield, z_slice_indx, y_slice_indx, mask)    
    # vector_x_up = vector_x_up[1:-1, 1:-1]
    # vector_y_up = vector_y_up[1:-1, 1:-1]
    # vector_x_low = vector_x_low[1:-1, 1:-1]
    # vector_y_low = vector_y_low[1:-1, 1:-1]
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # zup = zup[1:-1, 1:-1]
    # zlow = zlow[1:-1, 1:-1]
    # print("global min:", np.nanmin([np.nanmin(zup), np.nanmin(zlow)]), "global max:", np.nanmax([np.nanmax(zup), np.nanmax(zlow)]))
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\bar{{\nu}}_{{e}}}} \, / \,  n_{{\bar{{\nu}}_{{e}}}} | \, )$'
    # mincbar=min_cbar
    # maxcbar=max_cbar
    # mincbar=None
    # maxcbar=None
    # filename=f'./plots/{filepath.split('mesh_plt')[1].split('.h5')[0]}_fhatbaree'
    # plot_vector_map(zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)

    # with h5py.File('./plots/fbar_e.h5', 'w') as f:
    #     f.create_dataset('xup', data=Xc[1:-1,1:-1,16]/1e5)
    #     f.create_dataset('yup', data=Yc[1:-1,1:-1,16]/1e5)
    #     f.create_dataset('zup', data=zup)
    #     f.create_dataset('vector_x_up', data=vector_x_up)
    #     f.create_dataset('vector_y_up', data=vector_y_up)
    #     f.create_dataset('xlow', data=Xc[1:-1,48,1:-1]/1e5)
    #     f.create_dataset('ylow', data=Zc[1:-1,48,1:-1]/1e5)
    #     f.create_dataset('zlow', data=zlow)
    #     f.create_dataset('vector_x_low', data=vector_x_low)
    #     f.create_dataset('vector_y_low', data=vector_y_low)

    # # Muon antineutrino number flux density
    # vectorfield = np.array([np.array(alldata['Fx11_Rebar(1|ccm)']), # x component
    #                         np.array(alldata['Fy11_Rebar(1|ccm)']), # y component
    #                         np.array(alldata['Fz11_Rebar(1|ccm)'])]) # z component
    # zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low = compute_vector_field_components(vectorfield, time_indx, z_slice_indx, y_slice_indx, mask)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\bar{{\nu}}_{{\mu}}\bar{{\nu}}_{{\mu}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # mincbar=None
    # maxcbar=None
    # # mincbar=23.0
    # # maxcbar=26.6
    # filename=f'./plots/{finaldir}_fbaruu_t_{time_indx}'
    # plot_vector_map(zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)

    # # Tauon antineutrino number flux density
    # vectorfield = np.array([np.array(alldata['Fx22_Rebar(1|ccm)']), # x component
    #                         np.array(alldata['Fy22_Rebar(1|ccm)']), # y component
    #                         np.array(alldata['Fz22_Rebar(1|ccm)'])]) # z component
    # zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low = compute_vector_field_components(vectorfield, time_indx, z_slice_indx, y_slice_indx, mask)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\bar{{\nu}}_{{\tau}}\bar{{\nu}}_{{\tau}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # mincbar=None
    # maxcbar=None
    # # mincbar=23.0
    # # maxcbar=26.6
    # filename=f'./plots/{finaldir}_fbartt_t_{time_indx}'
    # plot_vector_map(zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)

    ###############################################
    # All heavies flux number density toguether
    ###############################################

    # # Heavy neutrino number flux density
    # fx = np.array(alldata['Fx11_Re(1|ccm)'])  # x component
    # fy = np.array(alldata['Fy11_Re(1|ccm)'])  # y component
    # fz = np.array(alldata['Fz11_Re(1|ccm)'])  # z component
    # n  = np.array(alldata['N11_Re(1|ccm)'])   # normalization
    # vectorfield = np.array([
    #     fx / n,
    #     fy / n,
    #     fz / n
    # ])
    # zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low = compute_vector_field_components(vectorfield, z_slice_indx, y_slice_indx, mask)
    # vector_x_up = vector_x_up[1:-1, 1:-1]
    # vector_y_up = vector_y_up[1:-1, 1:-1]
    # vector_x_low = vector_x_low[1:-1, 1:-1]
    # vector_y_low = vector_y_low[1:-1, 1:-1]
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # zup = zup[1:-1, 1:-1]
    # zlow = zlow[1:-1, 1:-1]
    # print("global min:", np.nanmin([np.nanmin(zup), np.nanmin(zlow)]), "global max:", np.nanmax([np.nanmax(zup), np.nanmax(zlow)]))
    # colorbarlabel = fr'$\log ( \, | \, \vec{{f}}_{{\nu_{{x}}}} \, / \, n_{{\nu_{{x}}}} \, | \, )$'
    # mincbar=min_cbar
    # maxcbar=max_cbar
    # mincbar=None
    # maxcbar=None
    # filename=f'./plots/{filepath.split('mesh_plt')[1].split('.h5')[0]}_fhatxx'
    # plot_vector_map(zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)

    # with h5py.File('./plots/f_x.h5', 'w') as f:
    #     f.create_dataset('xup', data=Xc[1:-1,1:-1,16]/1e5)
    #     f.create_dataset('yup', data=Yc[1:-1,1:-1,16]/1e5)
    #     f.create_dataset('zup', data=zup)
    #     f.create_dataset('vector_x_up', data=vector_x_up)
    #     f.create_dataset('vector_y_up', data=vector_y_up)
    #     f.create_dataset('xlow', data=Xc[1:-1,48,1:-1]/1e5)
    #     f.create_dataset('ylow', data=Zc[1:-1,48,1:-1]/1e5)
    #     f.create_dataset('zlow', data=zlow)
    #     f.create_dataset('vector_x_low', data=vector_x_low)
    #     f.create_dataset('vector_y_low', data=vector_y_low)

### Color maps with arrows: Neutrino and antineutrino off-diagonal fluxes

In [ ]:
x_slice_indx = 48 # cell number
y_slice_indx = 48 # cell number
z_slice_indx = 16 # cell number
time_indx    = -1 # last time step

# for time_indx in [1,2,3,4,5,6,7,8,9,10]:
for time_indx in [-1]:

    # if time_indx==10:
    if True:
        doshow = True
    else:
        doshow = False

    time_ms = np.array(alldata['t(s)'])[time_indx] * 1.0e3 # ms
    print('time_ms:', time_ms)

    ##############################################
    # Neutrino off-diagonal terms
    ##############################################

    # # Magnitud of electron-muon mixing neutrino flux number density
    # fmagup = np.sqrt(np.array(alldata['Fx01_Re(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2 +
    #                  np.array(alldata['Fx01_Im(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2 +
    #                  np.array(alldata['Fy01_Re(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2 +
    #                  np.array(alldata['Fy01_Im(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2 +
    #                  np.array(alldata['Fz01_Re(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2 +
    #                  np.array(alldata['Fz01_Im(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2)
    # zup = np.where(mask[1:-1, 1:-1, z_slice_indx], np.nan, fmagup)
    # fmaglow = np.sqrt(np.array(alldata['Fx01_Re(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2 +
    #                   np.array(alldata['Fx01_Im(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2 +
    #                   np.array(alldata['Fy01_Re(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2 +
    #                   np.array(alldata['Fy01_Im(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2 +
    #                   np.array(alldata['Fz01_Re(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2 +
    #                   np.array(alldata['Fz01_Im(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2)
    # zlow = np.where(mask[1:-1, y_slice_indx, 1:-1], np.nan, fmaglow)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\nu_{{e}}\nu_{{\mu}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # filename=f'./plots/{finaldir}_feu_t_{time_indx}'
    # mincbar=None
    # maxcbar=None
    # mincbar=21.9
    # maxcbar=25.3
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)

    # # Magnitud of electron-tauon mixing neutrino flux number density
    # fmagup = np.sqrt(np.array(alldata['Fx02_Re(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2 +
    #                  np.array(alldata['Fx02_Im(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2 +
    #                  np.array(alldata['Fy02_Re(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2 +
    #                  np.array(alldata['Fy02_Im(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2 +
    #                  np.array(alldata['Fz02_Re(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2 +
    #                  np.array(alldata['Fz02_Im(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2)
    # zup = np.where(mask[1:-1, 1:-1, z_slice_indx], np.nan, fmagup)
    # fmaglow = np.sqrt(np.array(alldata['Fx02_Re(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2 +
    #                   np.array(alldata['Fx02_Im(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2 +
    #                   np.array(alldata['Fy02_Re(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2 +
    #                   np.array(alldata['Fy02_Im(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2 +
    #                   np.array(alldata['Fz02_Re(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2 +
    #                   np.array(alldata['Fz02_Im(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2)
    # zlow = np.where(mask[1:-1, y_slice_indx, 1:-1], np.nan, fmaglow)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\nu_{{e}}\nu_{{\tau}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # filename=f'./plots/{finaldir}_fet_t_{time_indx}'
    # mincbar=None
    # maxcbar=None
    # mincbar=23.3
    # maxcbar=26.7
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)

    # # Magnitud of muon-tauon mixing neutrino flux number density
    # fmagup = np.sqrt(np.array(alldata['Fx12_Re(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2 +
    #                  np.array(alldata['Fx12_Im(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2 +
    #                  np.array(alldata['Fy12_Re(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2 +
    #                  np.array(alldata['Fy12_Im(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2 +
    #                  np.array(alldata['Fz12_Re(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2 +
    #                  np.array(alldata['Fz12_Im(1|ccm)'])[time_indx, 1:-1, 1:-1, z_slice_indx]**2)
    # zup = np.where(mask[1:-1, 1:-1, z_slice_indx], np.nan, fmagup)
    # fmaglow = np.sqrt(np.array(alldata['Fx12_Re(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2 +
    #                   np.array(alldata['Fx12_Im(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2 +
    #                   np.array(alldata['Fy12_Re(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2 +
    #                   np.array(alldata['Fy12_Im(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2 +
    #                   np.array(alldata['Fz12_Re(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2 +
    #                   np.array(alldata['Fz12_Im(1|ccm)'])[time_indx, 1:-1, y_slice_indx, 1:-1]**2)
    # zlow = np.where(mask[1:-1, y_slice_indx, 1:-1], np.nan, fmaglow)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\nu_{{\mu}}\nu_{{\tau}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # filename=f'./plots/{finaldir}_fut_t_{time_indx}'
    # mincbar=None
    # maxcbar=None
    # mincbar=16.8
    # maxcbar=20.2
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)

    ##############################################
    # Antineutrino off-diagonal terms
    ##############################################

    # # Magnitud of electron-muon mixing antineutrino flux number density
    # fmagup = np.sqrt(np.array(alldata['Fx01_Rebar(1|ccm)'])[time_indx, :, :, z_slice_indx]**2 +
    #                  np.array(alldata['Fx01_Imbar(1|ccm)'])[time_indx, :, :, z_slice_indx]**2 +
    #                  np.array(alldata['Fy01_Rebar(1|ccm)'])[time_indx, :, :, z_slice_indx]**2 +
    #                  np.array(alldata['Fy01_Imbar(1|ccm)'])[time_indx, :, :, z_slice_indx]**2 +
    #                  np.array(alldata['Fz01_Rebar(1|ccm)'])[time_indx, :, :, z_slice_indx]**2 +
    #                  np.array(alldata['Fz01_Imbar(1|ccm)'])[time_indx, :, :, z_slice_indx]**2)
    # zup = np.where(mask[:, :, z_slice_indx], np.nan, fmagup)
    # fmaglow = np.sqrt(np.array(alldata['Fx01_Rebar(1|ccm)'])[time_indx, :, y_slice_indx, :]**2 +
    #                   np.array(alldata['Fx01_Imbar(1|ccm)'])[time_indx, :, y_slice_indx, :]**2 +
    #                   np.array(alldata['Fy01_Rebar(1|ccm)'])[time_indx, :, y_slice_indx, :]**2 +
    #                   np.array(alldata['Fy01_Imbar(1|ccm)'])[time_indx, :, y_slice_indx, :]**2 +
    #                   np.array(alldata['Fz01_Rebar(1|ccm)'])[time_indx, :, y_slice_indx, :]**2 +
    #                   np.array(alldata['Fz01_Imbar(1|ccm)'])[time_indx, :, y_slice_indx, :]**2)
    # zlow = np.where(mask[:, y_slice_indx, :], np.nan, fmaglow)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\bar{{\nu}}_{{e}}\bar{{\nu}}_{{\mu}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # filename=f'./plots/{finaldir}_fbareu_t_{time_indx}'
    # mincbar=None
    # maxcbar=None
    # mincbar=21.9
    # maxcbar=24.9
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)

    # # Magnitud of electron-tauon mixing antineutrino flux number density
    # fmagup = np.sqrt(np.array(alldata['Fx02_Rebar(1|ccm)'])[time_indx, :, :, z_slice_indx]**2 +
    #                  np.array(alldata['Fx02_Imbar(1|ccm)'])[time_indx, :, :, z_slice_indx]**2 +
    #                  np.array(alldata['Fy02_Rebar(1|ccm)'])[time_indx, :, :, z_slice_indx]**2 +
    #                  np.array(alldata['Fy02_Imbar(1|ccm)'])[time_indx, :, :, z_slice_indx]**2 +
    #                  np.array(alldata['Fz02_Rebar(1|ccm)'])[time_indx, :, :, z_slice_indx]**2 +
    #                  np.array(alldata['Fz02_Imbar(1|ccm)'])[time_indx, :, :, z_slice_indx]**2)
    # zup = np.where(mask[:, :, z_slice_indx], np.nan, fmagup)
    # fmaglow = np.sqrt(np.array(alldata['Fx02_Rebar(1|ccm)'])[time_indx, :, y_slice_indx, :]**2 +
    #                   np.array(alldata['Fx02_Imbar(1|ccm)'])[time_indx, :, y_slice_indx, :]**2 +
    #                   np.array(alldata['Fy02_Rebar(1|ccm)'])[time_indx, :, y_slice_indx, :]**2 +
    #                   np.array(alldata['Fy02_Imbar(1|ccm)'])[time_indx, :, y_slice_indx, :]**2 +
    #                   np.array(alldata['Fz02_Rebar(1|ccm)'])[time_indx, :, y_slice_indx, :]**2 +
    #                   np.array(alldata['Fz02_Imbar(1|ccm)'])[time_indx, :, y_slice_indx, :]**2)
    # zlow = np.where(mask[:, y_slice_indx, :], np.nan, fmaglow)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\bar{{\nu}}_{{e}}\bar{{\nu}}_{{\tau}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # filename=f'./plots/{finaldir}_fbaret_t_{time_indx}'
    # mincbar=None
    # maxcbar=None
    # mincbar=23.3
    # maxcbar=26.3
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)

    # # Magnitud of muon-tauon mixing antineutrino flux number density
    # fmagup = np.sqrt(np.array(alldata['Fx12_Rebar(1|ccm)'])[time_indx, :, :, z_slice_indx]**2 +
    #                  np.array(alldata['Fx12_Imbar(1|ccm)'])[time_indx, :, :, z_slice_indx]**2 +
    #                  np.array(alldata['Fy12_Rebar(1|ccm)'])[time_indx, :, :, z_slice_indx]**2 +
    #                  np.array(alldata['Fy12_Imbar(1|ccm)'])[time_indx, :, :, z_slice_indx]**2 +
    #                  np.array(alldata['Fz12_Rebar(1|ccm)'])[time_indx, :, :, z_slice_indx]**2 +
    #                  np.array(alldata['Fz12_Imbar(1|ccm)'])[time_indx, :, :, z_slice_indx]**2)
    # zup = np.where(mask[:, :, z_slice_indx], np.nan, fmagup)
    # fmaglow = np.sqrt(np.array(alldata['Fx12_Rebar(1|ccm)'])[time_indx, :, y_slice_indx, :]**2 +
    #                   np.array(alldata['Fx12_Imbar(1|ccm)'])[time_indx, :, y_slice_indx, :]**2 +
    #                   np.array(alldata['Fy12_Rebar(1|ccm)'])[time_indx, :, y_slice_indx, :]**2 +
    #                   np.array(alldata['Fy12_Imbar(1|ccm)'])[time_indx, :, y_slice_indx, :]**2 +
    #                   np.array(alldata['Fz12_Rebar(1|ccm)'])[time_indx, :, y_slice_indx, :]**2 +
    #                   np.array(alldata['Fz12_Imbar(1|ccm)'])[time_indx, :, y_slice_indx, :]**2)
    # zlow = np.where(mask[:, y_slice_indx, :], np.nan, fmaglow)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\bar{{\nu}}_{{\mu}}\bar{{\nu}}_{{\tau}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # filename=f'./plots/{finaldir}_fbarut_t_{time_indx}'
    # mincbar=None
    # maxcbar=None
    # mincbar=16.1
    # maxcbar=19.7
    # plot_up_low(zup, zlow, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar,maxcbar, doshow=doshow, dosave=True)


In [ ]:
x_slice_indx = 96 # cell number
y_slice_indx = 96 # cell number
z_slice_indx = 32 # cell number
time_indx    = -1 # last time step

# for filepath in mesh_files_sorted[::100]:
for filepath in [mesh_files_sorted[20], mesh_files_sorted[70],mesh_files_sorted[-1]]:
# for filepath in mesh_files_sorted:
# for filepath in mesh_files_sorted[613:]:

    print(f'Processing file: {filepath}')

    alldata_h5 = h5py.File(filepath, 'r')
    alldata_dict = {key: np.array(alldata_h5[key]) for key in alldata_h5.keys()}
    alldata_h5.close()
    alldata = alldata_dict.copy()
    print("Keys in alldata_dict:", list(alldata.keys()))

    # if time_indx==0:
    if True:
        doshow = True
    else:
        doshow = False

    time_ms = np.array(alldata['t(s)']) * 1.0e3 # ms
    print("time_ms=", time_ms)

    ##############################################
    # Neutrino off-diagonal fluxes with arrows
    ##############################################

    # # Electron-muon mixing neutrino number flux density
    # vectorfield_re = np.array([np.array(alldata['Fx01_Re(1|ccm)']),  # x component
    #                            np.array(alldata['Fy01_Re(1|ccm)']),  # y component
    #                            np.array(alldata['Fz01_Re(1|ccm)'])]) # z component
    # zup_re, zlow_re, vector_x_up_re, vector_y_up_re, vector_x_low_re, vector_y_low_re = compute_vector_field_components(vectorfield_re, z_slice_indx, y_slice_indx, mask)
    # vectorfield_im = np.array([np.array(alldata['Fx01_Im(1|ccm)']),  # x component
    #                            np.array(alldata['Fy01_Im(1|ccm)']),  # y component
    #                            np.array(alldata['Fz01_Im(1|ccm)'])]) # z component
    # zup_im, zlow_im, vector_x_up_im, vector_y_up_im, vector_x_low_im, vector_y_low_im = compute_vector_field_components(vectorfield_im, z_slice_indx, y_slice_indx, mask)    
    # zup = np.sqrt(zup_re**2 + zup_im**2)
    # zlow = np.sqrt(zlow_re**2 + zlow_im**2)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # both_min = np.nanmin([np.nanmin(zlow), np.nanmin(zup)])
    # both_max = np.nanmax([np.nanmax(zlow), np.nanmax(zup)])
    # print(f"mincbar={both_min}")
    # print(f"maxcbar={both_max}")
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\nu_{{e}}\nu_{{\mu}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # mincbar=None
    # maxcbar=None
    # mincbar=21.488702793759654
    # maxcbar=26.72773471590992
    # filename=f'./plots/{filepath.split('mesh_plt')[1].split('.h5')[0]}_feu_t_{time_indx}'
    # plot_two_vector_map(zup, zlow, vector_x_up_re, vector_y_up_re, vector_x_up_im, vector_y_up_im, vector_x_low_re, vector_y_low_re, vector_x_low_im, vector_y_low_im, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)

    # with h5py.File(f'./plots/f_eu_{filepath.split('mesh_plt')[1].split('.h5')[0]}.h5', 'w') as f:
    #     f.create_dataset('xup', data=Xc[1:-1,1:-1,32]/1e5)
    #     f.create_dataset('yup', data=Yc[1:-1,1:-1,32]/1e5)
    #     f.create_dataset('zup', data=zup[1:-1, 1:-1])
    #     f.create_dataset('vector_x_up', data=vector_x_up_re[1:-1, 1:-1])
    #     f.create_dataset('vector_y_up', data=vector_y_up_re[1:-1, 1:-1])
    #     f.create_dataset('vector_x_up_im', data=vector_x_up_im[1:-1, 1:-1])
    #     f.create_dataset('vector_y_up_im', data=vector_y_up_im[1:-1, 1:-1])

    #     f.create_dataset('xlow', data=Xc[1:-1,96,1:-1]/1e5)
    #     f.create_dataset('ylow', data=Zc[1:-1,96,1:-1]/1e5)
    #     f.create_dataset('zlow', data=zlow[1:-1, 1:-1])
    #     f.create_dataset('vector_x_low', data=vector_x_low_re[1:-1, 1:-1])
    #     f.create_dataset('vector_y_low', data=vector_y_low_re[1:-1, 1:-1])
    #     f.create_dataset('vector_x_low_im', data=vector_x_low_im[1:-1, 1:-1])
    #     f.create_dataset('vector_y_low_im', data=vector_y_low_im[1:-1, 1:-1])

    # # Electron-taun mixing neutrino number flux density
    # vectorfield_re = np.array([np.array(alldata['Fx02_Re(1|ccm)']),  # x component
    #                            np.array(alldata['Fy02_Re(1|ccm)']),  # y component
    #                            np.array(alldata['Fz02_Re(1|ccm)'])]) # z component
    # zup_re, zlow_re, vector_x_up_re, vector_y_up_re, vector_x_low_re, vector_y_low_re = compute_vector_field_components(vectorfield_re, time_indx, z_slice_indx, y_slice_indx, mask)
    # vectorfield_im = np.array([np.array(alldata['Fx02_Im(1|ccm)']),  # x component
    #                            np.array(alldata['Fy02_Im(1|ccm)']),  # y component
    #                            np.array(alldata['Fz02_Im(1|ccm)'])]) # z component
    # zup_im, zlow_im, vector_x_up_im, vector_y_up_im, vector_x_low_im, vector_y_low_im = compute_vector_field_components(vectorfield_im, time_indx, z_slice_indx, y_slice_indx, mask)    
    # zup = np.sqrt(zup_re**2 + zup_im**2)
    # zlow = np.sqrt(zlow_re**2 + zlow_im**2)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # print("mincbar=", np.nanmin(zlow), "\nmaxcbar=", np.nanmax(zlow))
    # both_min = np.nanmin([np.nanmin(zlow), np.nanmin(zup)])
    # both_max = np.nanmax([np.nanmax(zlow), np.nanmax(zup)])
    # print(f"mincbar={both_min}")
    # print(f"maxcbar={both_max}")
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\nu_{{e}}\nu_{{\tau}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # mincbar=None
    # maxcbar=None
    # mincbar= 23.15040148631209 
    # maxcbar= 28.125315951614354
    # filename=f'./plots/{finaldir}_fet_t_{time_indx}'
    # plot_two_vector_map(zup, zlow, vector_x_up_re, vector_y_up_re, vector_x_up_im, vector_y_up_im, vector_x_low_re, vector_y_low_re, vector_x_low_im, vector_y_low_im, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)

    # # Muon-taun mixing neutrino number flux density
    # vectorfield_re = np.array([np.array(alldata['Fx12_Re(1|ccm)']),  # x component
    #                            np.array(alldata['Fy12_Re(1|ccm)']),  # y component
    #                            np.array(alldata['Fz12_Re(1|ccm)'])]) # z component
    # zup_re, zlow_re, vector_x_up_re, vector_y_up_re, vector_x_low_re, vector_y_low_re = compute_vector_field_components(vectorfield_re, time_indx, z_slice_indx, y_slice_indx, mask)
    # vectorfield_im = np.array([np.array(alldata['Fx12_Im(1|ccm)']),  # x component
    #                            np.array(alldata['Fy12_Im(1|ccm)']),  # y component
    #                            np.array(alldata['Fz12_Im(1|ccm)'])]) # z component
    # zup_im, zlow_im, vector_x_up_im, vector_y_up_im, vector_x_low_im, vector_y_low_im = compute_vector_field_components(vectorfield_im, time_indx, z_slice_indx, y_slice_indx, mask)    
    # zup = np.sqrt(zup_re**2 + zup_im**2)
    # zlow = np.sqrt(zlow_re**2 + zlow_im**2)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # both_min = np.nanmin([np.nanmin(zlow), np.nanmin(zup)])
    # both_max = np.nanmax([np.nanmax(zlow), np.nanmax(zup)])
    # print(f"mincbar={both_min}")
    # print(f"maxcbar={both_max}")
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\nu_{{\mu}}\nu_{{\tau}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # mincbar=None
    # maxcbar=None
    # # mincbar=21.632973218228464
    # # maxcbar=28.144348096841764
    # filename=f'./plots/{finaldir}_fut_t_{time_indx}'
    # plot_two_vector_map(zup, zlow, vector_x_up_re, vector_y_up_re, vector_x_up_im, vector_y_up_im, vector_x_low_re, vector_y_low_re, vector_x_low_im, vector_y_low_im, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)

    # # Electron-muon mixing antineutrino number flux density
    # vectorfield_re = np.array([np.array(alldata['Fx01_Rebar(1|ccm)']),  # x component
    #                            np.array(alldata['Fy01_Rebar(1|ccm)']),  # y component
    #                            np.array(alldata['Fz01_Rebar(1|ccm)'])]) # z component
    # zup_re, zlow_re, vector_x_up_re, vector_y_up_re, vector_x_low_re, vector_y_low_re = compute_vector_field_components(vectorfield_re, time_indx, z_slice_indx, y_slice_indx, mask)
    # vectorfield_im = np.array([np.array(alldata['Fx01_Imbar(1|ccm)']),  # x component
    #                            np.array(alldata['Fy01_Imbar(1|ccm)']),  # y component
    #                            np.array(alldata['Fz01_Imbar(1|ccm)'])]) # z component
    # zup_im, zlow_im, vector_x_up_im, vector_y_up_im, vector_x_low_im, vector_y_low_im = compute_vector_field_components(vectorfield_im, time_indx, z_slice_indx, y_slice_indx, mask)    
    # zup = np.sqrt(zup_re**2 + zup_im**2)
    # zlow = np.sqrt(zlow_re**2 + zlow_im**2)
    # zup = np.log10(zup)
    # zlow = np.log10(zlow)
    # both_min = np.nanmin([np.nanmin(zlow), np.nanmin(zup)])
    # both_max = np.nanmax([np.nanmax(zlow), np.nanmax(zup)])
    # print(f"mincbar={both_min}")
    # print(f"maxcbar={both_max}")
    # colorbarlabel = fr'$\log (\,|\,\vec{{f}}_{{\bar{{\nu}}_{{e}}\bar{{\nu}}_{{\mu}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # mincbar=None
    # maxcbar=None
    # # mincbar=21.488702793759654
    # # maxcbar=26.72773471590992
    # filename=f'./plots/{finaldir}_feubar_t_{time_indx}'
    # plot_two_vector_map(zup, zlow, vector_x_up_re, vector_y_up_re, vector_x_up_im, vector_y_up_im, vector_x_low_re, vector_y_low_re, vector_x_low_im, vector_y_low_im, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)


In [ ]:
x_slice_indx = 48 # cell number
y_slice_indx = 48 # cell number
z_slice_indx = 16 # cell number
time_indx    = -1 # last time step

# for time_indx in [1,2,3,4,5,6,7,8,9,10]:
for time_indx in [-1]:

    # if time_indx==10:
    if True:
        doshow = True
    else:
        doshow = False

    time_ms = np.array(alldata['t(s)'])[time_indx] * 1.0e3 # ms
    print('time_ms:', time_ms)

    ##############################################
    # Neutrino off-diagonal densities with arrows as complex phase
    ##############################################

    # # # Electron-muon mixing neutrino number density with arrows as complex phase
    # thetaphase   = np.arctan2(alldata['F01_Im(1|ccm)'], alldata['F01_Re(1|ccm)'])
    # vector_x_up  = np.cos(thetaphase)[time_indx, :, :, z_slice_indx]
    # vector_y_up  = np.sin(thetaphase)[time_indx, :, :, z_slice_indx]
    # vector_x_low = np.cos(thetaphase)[time_indx, :, y_slice_indx, :]
    # vector_y_low = np.sin(thetaphase)[time_indx, :, y_slice_indx, :]
    # vector_x_up = np.where(mask[:, :, z_slice_indx], np.nan, vector_x_up)
    # vector_y_up = np.where(mask[:, :, z_slice_indx], np.nan, vector_y_up)
    # vector_x_low = np.where(mask[:, y_slice_indx, :], np.nan, vector_x_low)
    # vector_y_low = np.where(mask[:, y_slice_indx, :], np.nan, vector_y_low)
    # zup = np.sqrt(np.array(alldata['F01_Re(1|ccm)'])[time_indx, :, :, z_slice_indx]**2 +
    #               np.array(alldata['F01_Im(1|ccm)'])[time_indx, :, :, z_slice_indx]**2)
    # zlow = np.sqrt(np.array(alldata['F01_Re(1|ccm)'])[time_indx, :, y_slice_indx, :]**2 +
    #                 np.array(alldata['F01_Im(1|ccm)'])[time_indx, :, y_slice_indx, :]**2)
    # zup  = np.log10(zup)
    # zlow = np.log10(zlow)
    # zlow = np.where(mask[:, y_slice_indx, :], np.nan, zlow)
    # zup = np.where(mask[:, :, z_slice_indx], np.nan, zup)
    # colorbarlabel = fr'$\log (\,|\,n_{{\nu_{{e}}\nu_{{\mu}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # mincbar=None
    # maxcbar=None
    # # mincbar=21.9
    # # maxcbar=26.5
    # filename=f'./plots/{finaldir}_neu_t_{time_indx}'
    # plot_vector_map(zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)

    # # # Electron-tauon mixing neutrino number density with arrows as complex phase
    # thetaphase   = np.arctan2(alldata['F02_Im(1|ccm)'], alldata['F02_Re(1|ccm)'])
    # vector_x_up  = np.cos(thetaphase)[time_indx, :, :, z_slice_indx]
    # vector_y_up  = np.sin(thetaphase)[time_indx, :, :, z_slice_indx]
    # vector_x_low = np.cos(thetaphase)[time_indx, :, y_slice_indx, :]
    # vector_y_low = np.sin(thetaphase)[time_indx, :, y_slice_indx, :]
    # vector_x_up = np.where(mask[:, :, z_slice_indx], np.nan, vector_x_up)
    # vector_y_up = np.where(mask[:, :, z_slice_indx], np.nan, vector_y_up)
    # vector_x_low = np.where(mask[:, y_slice_indx, :], np.nan, vector_x_low)
    # vector_y_low = np.where(mask[:, y_slice_indx, :], np.nan, vector_y_low)
    # zup = np.sqrt(np.array(alldata['F02_Re(1|ccm)'])[time_indx, :, :, z_slice_indx]**2 +
    #               np.array(alldata['F02_Im(1|ccm)'])[time_indx, :, :, z_slice_indx]**2)
    # zlow = np.sqrt(np.array(alldata['F02_Re(1|ccm)'])[time_indx, :, y_slice_indx, :]**2 +
    #                np.array(alldata['F02_Im(1|ccm)'])[time_indx, :, y_slice_indx, :]**2)
    # zup  = np.log10(zup)
    # zlow = np.log10(zlow)
    # zlow = np.where(mask[:, y_slice_indx, :], np.nan, zlow)
    # zup = np.where(mask[:, :, z_slice_indx], np.nan, zup)
    # colorbarlabel = fr'$\log (\,|\,n_{{\nu_{{e}}\nu_{{\tau}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # mincbar=None
    # maxcbar=None
    # # mincbar=21.9
    # # maxcbar=26.5
    # filename=f'./plots/{finaldir}_net_t_{time_indx}'
    # plot_vector_map(zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)

    # # # Muon-tauon mixing neutrino number density with arrows as complex phase
    # thetaphase   = np.arctan2(alldata['F12_Im(1|ccm)'], alldata['F12_Re(1|ccm)'])
    # vector_x_up  = np.cos(thetaphase)[time_indx, :, :, z_slice_indx]
    # vector_y_up  = np.sin(thetaphase)[time_indx, :, :, z_slice_indx]
    # vector_x_low = np.cos(thetaphase)[time_indx, :, y_slice_indx, :]
    # vector_y_low = np.sin(thetaphase)[time_indx, :, y_slice_indx, :]
    # vector_x_up = np.where(mask[:, :, z_slice_indx], np.nan, vector_x_up)
    # vector_y_up = np.where(mask[:, :, z_slice_indx], np.nan, vector_y_up)
    # vector_x_low = np.where(mask[:, y_slice_indx, :], np.nan, vector_x_low)
    # vector_y_low = np.where(mask[:, y_slice_indx, :], np.nan, vector_y_low)
    # zup = np.sqrt(np.array(alldata['F12_Re(1|ccm)'])[time_indx, :, :, z_slice_indx]**2 +
    #               np.array(alldata['F12_Im(1|ccm)'])[time_indx, :, :, z_slice_indx]**2)
    # zlow = np.sqrt(np.array(alldata['F12_Re(1|ccm)'])[time_indx, :, y_slice_indx, :]**2 +
    #                np.array(alldata['F12_Im(1|ccm)'])[time_indx, :, y_slice_indx, :]**2)
    # zup  = np.log10(zup)
    # zlow = np.log10(zlow)
    # zlow = np.where(mask[:, y_slice_indx, :], np.nan, zlow)
    # zup = np.where(mask[:, :, z_slice_indx], np.nan, zup)
    # colorbarlabel = fr'$\log (\,|\,n_{{\nu_{{\mu}}\nu_{{\tau}}}}\,|\,[\mathrm{{cm}}^{{-3}}]\,)$'
    # mincbar=None
    # maxcbar=None
    # # mincbar=21.9
    # # maxcbar=26.5
    # filename=f'./plots/{finaldir}_nut_t_{time_indx}'
    # plot_vector_map(zup, zlow, vector_x_up, vector_y_up, vector_x_low, vector_y_low, time_ms, colorbarlabel, z_slice_indx, y_slice_indx, filename, mincbar, maxcbar, doshow=doshow, dosave=True)


### CREATE MOVIES

In [ ]:
# directory = './plots'
# frame_rate = 3 # images per second
# output_filename = './plots/log_nu_et_xz.mp4'
# image_files = sorted(glob.glob(os.path.join(directory, '*xz.png')))
# image_files = sorted(image_files, key=lambda x: int(x.split('_')[4]))
# print('image_files=',image_files)
# image_files= ['./plots/att_1.0e-05_net_t_1_xz.png','./plots/att_1.0e-05_net_t_1_xz.png','./plots/att_1.0e-05_net_t_1_xz.png','./plots/att_1.0e-05_net_t_2_xz.png', './plots/att_1.0e-05_net_t_3_xz.png', './plots/att_1.0e-05_net_t_4_xz.png', './plots/att_1.0e-05_net_t_5_xz.png', './plots/att_1.0e-05_net_t_6_xz.png', './plots/att_1.0e-05_net_t_7_xz.png', './plots/att_1.0e-05_net_t_8_xz.png', './plots/att_1.0e-05_net_t_9_xz.png','./plots/att_1.0e-05_net_t_10_xz.png','./plots/att_1.0e-05_net_t_10_xz.png','./plots/att_1.0e-05_net_t_10_xz.png']

# directory = './plots'
# frame_rate = 3 # images per second
# output_filename = './plots/log_nu_et_xy.mp4'
# image_files = sorted(glob.glob(os.path.join(directory, '*xy.png')))
# image_files = sorted(image_files, key=lambda x: int(x.split('_')[4]))
# print('image_files=',image_files)
# image_files= ['./plots/att_1.0e-05_net_t_1_xy.png','./plots/att_1.0e-05_net_t_1_xy.png','./plots/att_1.0e-05_net_t_1_xy.png', './plots/att_1.0e-05_net_t_2_xy.png', './plots/att_1.0e-05_net_t_3_xy.png', './plots/att_1.0e-05_net_t_4_xy.png', './plots/att_1.0e-05_net_t_5_xy.png', './plots/att_1.0e-05_net_t_6_xy.png', './plots/att_1.0e-05_net_t_7_xy.png', './plots/att_1.0e-05_net_t_8_xy.png', './plots/att_1.0e-05_net_t_9_xy.png', './plots/att_1.0e-05_net_t_10_xy.png', './plots/att_1.0e-05_net_t_10_xy.png', './plots/att_1.0e-05_net_t_10_xy.png']

# directory = './plots'
# frame_rate = 1 # images per second
# output_filename = './plots/log_re_f_eu_xz.mp4'
# image_files = sorted(glob.glob(os.path.join(directory, '*xz.png')))
# image_files = sorted(image_files, key=lambda x: int(x.split('_')[4]))
# print('image_files=',image_files)
# image_files= ['./plots/att_1.0e-05_refeu_t_1_xz.png', './plots/att_1.0e-05_refeu_t_1_xz.png', './plots/att_1.0e-05_refeu_t_2_xz.png', './plots/att_1.0e-05_refeu_t_3_xz.png', './plots/att_1.0e-05_refeu_t_4_xz.png', './plots/att_1.0e-05_refeu_t_5_xz.png', './plots/att_1.0e-05_refeu_t_6_xz.png', './plots/att_1.0e-05_refeu_t_7_xz.png', './plots/att_1.0e-05_refeu_t_8_xz.png', './plots/att_1.0e-05_refeu_t_9_xz.png', './plots/att_1.0e-05_refeu_t_10_xz.png', './plots/att_1.0e-05_refeu_t_10_xz.png', './plots/att_1.0e-05_refeu_t_10_xz.png']

# directory = './plots'
# frame_rate = 1 # images per second
# output_filename = './plots/log_re_f_eu_xy.mp4'
# image_files = sorted(glob.glob(os.path.join(directory, '*xy.png')))
# image_files = sorted(image_files, key=lambda x: int(x.split('_')[4]))
# print('image_files=',image_files)
# image_files= ['./plots/att_1.0e-05_refeu_t_1_xy.png', './plots/att_1.0e-05_refeu_t_1_xy.png', './plots/att_1.0e-05_refeu_t_2_xy.png', './plots/att_1.0e-05_refeu_t_3_xy.png', './plots/att_1.0e-05_refeu_t_4_xy.png', './plots/att_1.0e-05_refeu_t_5_xy.png', './plots/att_1.0e-05_refeu_t_6_xy.png', './plots/att_1.0e-05_refeu_t_7_xy.png', './plots/att_1.0e-05_refeu_t_8_xy.png', './plots/att_1.0e-05_refeu_t_9_xy.png', './plots/att_1.0e-05_refeu_t_10_xy.png', './plots/att_1.0e-05_refeu_t_10_xy.png', './plots/att_1.0e-05_refeu_t_10_xy.png']

# directory = './plots'
# frame_rate = 3 # images per second
# output_filename = './plots/log_im_f_et_xy.mp4'
# image_files = sorted(glob.glob(os.path.join(directory, 'att_1.0e-05_imfet_t_*xy.png')))
# image_files = sorted(image_files, key=lambda x: int(x.split('_')[4]))
# print('image_files=',image_files)
# image_files= ['./plots/att_1.0e-05_imfet_t_1_xy.png','./plots/att_1.0e-05_imfet_t_1_xy.png','./plots/att_1.0e-05_imfet_t_1_xy.png', './plots/att_1.0e-05_imfet_t_2_xy.png', './plots/att_1.0e-05_imfet_t_3_xy.png', './plots/att_1.0e-05_imfet_t_4_xy.png', './plots/att_1.0e-05_imfet_t_5_xy.png', './plots/att_1.0e-05_imfet_t_6_xy.png', './plots/att_1.0e-05_imfet_t_7_xy.png', './plots/att_1.0e-05_imfet_t_8_xy.png', './plots/att_1.0e-05_imfet_t_9_xy.png', './plots/att_1.0e-05_imfet_t_10_xy.png', './plots/att_1.0e-05_imfet_t_10_xy.png', './plots/att_1.0e-05_imfet_t_10_xy.png']


# directory = './plots'
# frame_rate = 3 # images per second
# output_filename = './plots/log_re_f_eu_xy.mp4'
# image_files = sorted(glob.glob(os.path.join(directory, 'att_1.0e-05_refet_t_*xy.png')))
# image_files = sorted(image_files, key=lambda x: int(x.split('_')[4]))
# print('image_files=',image_files)
# image_files= ['./plots/att_1.0e-05_refet_t_1_xy.png','./plots/att_1.0e-05_refet_t_1_xy.png','./plots/att_1.0e-05_refet_t_1_xy.png', './plots/att_1.0e-05_refet_t_2_xy.png', './plots/att_1.0e-05_refet_t_3_xy.png', './plots/att_1.0e-05_refet_t_4_xy.png', './plots/att_1.0e-05_refet_t_5_xy.png', './plots/att_1.0e-05_refet_t_6_xy.png', './plots/att_1.0e-05_refet_t_7_xy.png', './plots/att_1.0e-05_refet_t_8_xy.png', './plots/att_1.0e-05_refet_t_9_xy.png', './plots/att_1.0e-05_refet_t_10_xy.png', './plots/att_1.0e-05_refet_t_10_xy.png', './plots/att_1.0e-05_refet_t_10_xy.png']



# directory = './plots'
# frame_rate = 3 # images per second
# output_filename = './plots/log_im_f_et_xz.mp4'
# image_files = sorted(glob.glob(os.path.join(directory, 'att_1.0e-05_imfet_t_*xz.png')))
# image_files = sorted(image_files, key=lambda x: int(x.split('_')[4]))
# print('image_files=',image_files)
# image_files= ['./plots/att_1.0e-05_imfet_t_1_xz.png', './plots/att_1.0e-05_imfet_t_1_xz.png', './plots/att_1.0e-05_imfet_t_1_xz.png', './plots/att_1.0e-05_imfet_t_2_xz.png', './plots/att_1.0e-05_imfet_t_3_xz.png', './plots/att_1.0e-05_imfet_t_4_xz.png', './plots/att_1.0e-05_imfet_t_5_xz.png', './plots/att_1.0e-05_imfet_t_6_xz.png', './plots/att_1.0e-05_imfet_t_7_xz.png', './plots/att_1.0e-05_imfet_t_8_xz.png', './plots/att_1.0e-05_imfet_t_9_xz.png', './plots/att_1.0e-05_imfet_t_10_xz.png', './plots/att_1.0e-05_imfet_t_10_xz.png', './plots/att_1.0e-05_imfet_t_10_xz.png']



# directory = './plots'
# frame_rate = 3 # images per second
# output_filename = './plots/log_re_f_et_xy.mp4'
# image_files = sorted(glob.glob(os.path.join(directory, 'att_1.0e-05_refet_t_*xy.png')))
# image_files = sorted(image_files, key=lambda x: int(x.split('_')[4]))
# print('image_files=',image_files)
# image_files= ['./plots/att_1.0e-05_refet_t_1_xy.png','./plots/att_1.0e-05_refet_t_1_xy.png','./plots/att_1.0e-05_refet_t_1_xy.png', './plots/att_1.0e-05_refet_t_2_xy.png', './plots/att_1.0e-05_refet_t_3_xy.png', './plots/att_1.0e-05_refet_t_4_xy.png', './plots/att_1.0e-05_refet_t_5_xy.png', './plots/att_1.0e-05_refet_t_6_xy.png', './plots/att_1.0e-05_refet_t_7_xy.png', './plots/att_1.0e-05_refet_t_8_xy.png', './plots/att_1.0e-05_refet_t_9_xy.png', './plots/att_1.0e-05_refet_t_10_xy.png', './plots/att_1.0e-05_refet_t_10_xy.png', './plots/att_1.0e-05_refet_t_10_xy.png']

# directory = './plots'
# frame_rate = 3 # images per second
# output_filename = './plots/log_re_f_et_xz.mp4'
# output_filename = './plots/log_re_f_et_arrow_xz.mp4'
# output_filename = './plots/log_re_f_et_arrow_xy.mp4'
# # image_files = sorted(glob.glob(os.path.join(directory, '1506_ppEb_neu_t_*xz.png')))
# image_files = sorted(glob.glob(os.path.join(directory, '1506_ppEb_feu_t_*xy.png')))
# image_files = sorted(image_files, key=lambda x: int(x.split('_')[4]))
# print('image_files=',image_files)
# # image_files= ['./plots/1506_ppEb_neu_t_-1_xz.png', './plots/1506_ppEb_neu_t_0_xz.png', './plots/1506_ppEb_neu_t_1_xz.png', './plots/1506_ppEb_neu_t_2_xz.png', './plots/1506_ppEb_neu_t_3_xz.png', './plots/1506_ppEb_neu_t_4_xz.png', './plots/1506_ppEb_neu_t_5_xz.png', './plots/1506_ppEb_neu_t_6_xz.png', './plots/1506_ppEb_neu_t_7_xz.png', './plots/1506_ppEb_neu_t_8_xz.png', './plots/1506_ppEb_neu_t_9_xz.png', './plots/1506_ppEb_neu_t_10_xz.png', './plots/1506_ppEb_neu_t_11_xz.png', './plots/1506_ppEb_neu_t_12_xz.png', './plots/1506_ppEb_neu_t_13_xz.png', './plots/1506_ppEb_neu_t_14_xz.png', './plots/1506_ppEb_neu_t_15_xz.png', './plots/1506_ppEb_neu_t_16_xz.png', './plots/1506_ppEb_neu_t_17_xz.png', './plots/1506_ppEb_neu_t_18_xz.png', './plots/1506_ppEb_neu_t_19_xz.png', './plots/1506_ppEb_neu_t_20_xz.png']

# directory = './plots'
# frame_rate = 3 # images per second
# output_filename = './plots/nee_xz.mp4'
# image_files = sorted(glob.glob(os.path.join(directory, '*_nee_xz.png')))
# image_files = sorted(image_files, key=lambda x: int(x.split('/')[-1].split('_')[0]))
# print('image_files=',image_files)

# directory = './plots'
# frame_rate = 3 # images per second
# output_filename = './plots/nee_xy.mp4'
# image_files = sorted(glob.glob(os.path.join(directory, '*_nee_xy.png')))
# image_files = sorted(image_files, key=lambda x: int(x.split('/')[-1].split('_')[0]))
# print('image_files=',image_files)

directory = './plots'
frame_rate = 10 # images per second
output_filename = './plots/low_limits_delta_rho_eu_xy.mp4'
image_files = sorted(glob.glob(os.path.join(directory, './*_delta_rho_eu_xy.png')))
image_files = sorted(image_files, key=lambda x: int(x.split('/')[-1].split('_')[0]))
print('image_files=',image_files)

# directory = './plots'
# frame_rate = 3 # images per second
# output_filename = './plots/nbaree_xy.mp4'
# image_files = sorted(glob.glob(os.path.join(directory, '*_nbaree_xy.png')))
# image_files = sorted(image_files, key=lambda x: int(x.split('/')[-1].split('_')[0]))
# print('image_files=',image_files)

if not image_files:
    raise ValueError(f"No images found in directory: {directory}")

# Read the first image to get the dimensions
frame = cv2.imread(image_files[0])
height, width, layers = frame.shape

# Scale up the resolution
scale_factor = 2  # Increase resolution by a factor of 2
new_width = width * scale_factor
new_height = height * scale_factor

# Define the codec and create VideoWriter object
video = cv2.VideoWriter(output_filename, cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, (new_width, new_height))

for image_file in image_files:
    # Read and resize each frame
    frame = cv2.imread(image_file)
    resized_frame = cv2.resize(frame, (new_width, new_height), interpolation=cv2.INTER_CUBIC)
    video.write(resized_frame)

# Release the video writer object

video.release()